# Import libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')
import psycopg2
from connectionsbyclassjava import *
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
import psutil
import kaleido
import plotly.io as pio
#pio.kaleido.scope.default_format = "svg"
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 2000)

# Define ETL functions

## MIMIC & eICU connections

In [3]:
def connection_lcp(db):
    
    if db == 'eicu':
        
        db_con = psycopg2.connect(
            dbname='eicu', user='postgres', host='db', password='supersecretpassword', port='5432')
        
    else:
        
        db_con = psycopg2.connect(
            dbname='mimic', user='postgres', host='db', password='supersecretpassword', port='5432')
        
    return db_con
        

## Patients with age >=16 & ICU LOS > 1day

In [6]:
def age_los_filter(db):
    
    if db == 'eicu':
        
        conn = connection_lcp(db)
        
        patients = pd.read_sql("""
                                SELECT 
                                patientunitstayid,
                                gender,
                                age,
                                unitDischargeOffset,
                                hospitalDischargeOffset,
                                hospitalDischargeStatus,
                                'eicu' AS db
                                FROM eicu.patient
                                WHERE unitDischargeOffset > 1440  
                                AND age <> ''
                                """, conn)
        
        conn.close()
        
        patients['age'] = np.where(patients['age'].str.contains('>'),'90',patients['age'])
        patients['age'] = patients['age'].astype(int)
        patients = patients[patients['age']>=16]
        patients.reset_index(drop=True, inplace=True)
        
        print ('eicu patients >= 16 and ICU LOS > 1 day: ', patients['patientunitstayid'].nunique())
        
    elif db == 'mimic':
        
        conn = connection_lcp(db)
        
        patients = pd.read_sql("""
                            SELECT 
                            icustays.icustay_id,
                            icustays.intime,
                            icustays.los, 
                            DATE_PART('year', icustays.intime) - DATE_PART('year', patients.dob) AS age,
                            admissions.dischtime,
                            admissions.admittime,
                            admissions.hadm_id,
                            admissions.hospital_expire_flag,
                            patients.subject_id,
                            patients.gender,
                            'mimic' AS db
                            FROM mimiciii.icustays icustays
                            INNER JOIN mimiciii.admissions admissions ON icustays.hadm_id = admissions.hadm_id
                            INNER JOIN mimiciii.patients patients ON admissions.subject_id = patients.subject_id
                            WHERE icustays.los > 1
                            """, conn)
        
        conn.close()
        
        patients['age'] = np.where(patients['age']>200, 90, patients['age']).astype(int)
        patients = patients[patients['age']>=16]
        patients['hosp_los'] = round((patients['dischtime'] - patients['admittime']).dt.total_seconds()/86400,1)
        patients.drop(['dischtime','admittime'], axis=1, inplace=True)
        patients.reset_index(drop=True, inplace=True)
        
        print ('mimic patients >= 16 and ICU LOS > 1 day: ', patients['icustay_id'].nunique())
        
    else:

        
        icupatients = ConnectionJava(hospital='JX').data_from_both_connections(
            """
                SELECT DISTINCT
                PatientID
                FROM PV_AdminWards
                WHERE AdminWardName = 'MEDICINA INTENSIVA'
                AND AdminWardStartTime > '2014-01-01'
                AND AdminWardEndTime < '2020-01-01'
            """)
        
        patlist = tuple(set(icupatients['PatientID']))
        
        print('total patients in hj23: ', icupatients['PatientID'].nunique())
        
        del icupatients

        patients = ConnectionJava(hospital='JX').data_from_both_connections(
            f"""
                SELECT 
                PatientID,
                PatientCode,
                AdmissionAgeInCalYears,
                PatientSex AS gender,
                ICU_days,
                AdmTime,
                'hj23' AS db
                FROM PV_PatientList
                WHERE PatientID IN {patlist}
                AND ICU_days > 1
            """)
        
        patients.rename({'AdmissionAgeInCalYears':'age'}, axis=1, inplace=True)
        
        patients = patients[patients['age']>=16]
        
        patlist = tuple(set(patients['PatientCode']))
    
        oracleiter = np.ceil(len(patlist)/1000)
        episodes_splitted = np.array_split(patlist, oracleiter)

        del oracleiter

        hospdis = pd.DataFrame()

        for episodes in episodes_splitted:

            episodes = list(filter(None, episodes)) 
    
            episodes = tuple(episodes)

            hospinfo = ConnectionJava(hospital='JX').chunked_read(
                f"""
                    SELECT
                    EPISODI,
                    CLASE_ALTA,
                    DIES_HOSPITALITZACIO
                    FROM DWFACTJX.HOSPITALITZACIO 
                    WHERE EPISODI IN {episodes}
                """, production=None)
            
            hospdis = pd.concat([hospdis, hospinfo])
            
        del episodes_splitted
    
        list_of_numbers = tuple(['0','1','2','3','4','5','6','7','8','9'])
        hospdis = hospdis[hospdis['EPISODI'].str.startswith(list_of_numbers)]
        hospdis = hospdis.rename(
            {'EPISODI':'PatientCode','CLASE_ALTA':'hosp_dis','DIES_HOSPITALITZACIO':'hosp_los'}, axis=1)
        hospdis['hosp_dis'] = np.where(hospdis['hosp_dis']=='6','EXPIRED','ALIVE')
    
        patients = pd.merge(patients, hospdis, on='PatientCode', how='inner')
        patients.drop_duplicates(inplace=True)
        patients.reset_index(drop=True, inplace=True)
        
        del patlist, hospinfo, hospdis, list_of_numbers
        
        print ('hj23 patients >= 16 and ICU LOS > 1 day: ', patients['PatientID'].nunique())
    
    return patients

## Patients with SIS and create SIS quartile

In [8]:
def sis(df):
    
    db = df['db'][0]
    
    if db == 'eicu':
        
        patlist = tuple(set(df['patientunitstayid']))
        
        conn = connection_lcp(db)
        
        sis = pd.read_sql(f"""
                            SELECT 
                            patientunitstayid,
                            apacheScore AS sis
                            FROM eicu.apachePatientResult
                            WHERE patientunitstayid IN {patlist} 
                            AND apacheScore > 0
                            """, conn)
        
        conn.close()
        
        sis_g = sis.groupby('patientunitstayid', as_index=False).agg({'sis':'last'})
        sis_g['sis_q'] = pd.qcut(sis_g['sis'], 4, labels=False) + 1
        df = pd.merge(df, sis_g, on='patientunitstayid', how='right')
        df.reset_index(drop=True, inplace=True)
        
        del db, patlist, sis, sis_g

        print ('eicu patients >= 16 and ICU LOS > 1 day with SIS: ', df['patientunitstayid'].nunique())
        
    elif db == 'mimic':
        
        patlist = tuple(set(df['icustay_id']))
        
        conn = connection_lcp(db)
        
        sis = pd.read_sql(f"""
                            SELECT 
                            icustay_id,
                            oasis AS sis
                            FROM mimiciii.oasis
                            WHERE icustay_id IN {patlist} 
                            AND oasis > 0
                            """, conn)
        
        conn.close()
        
        sis_g = sis.groupby('icustay_id', as_index=False).agg({'sis':'last'})
        sis_g['sis_q'] = pd.qcut(sis_g['sis'], 4, labels=False) + 1
        df = pd.merge(df, sis_g, on='icustay_id', how='right')
        df.reset_index(drop=True, inplace=True)
        
        del db, patlist, sis, sis_g
        
        print ('mimic patients >= 16 and ICU LOS > 1 day with SIS: ', df['icustay_id'].nunique())
        
    else:
        
        patlist = tuple(set(df['PatientID']))

        sis = ConnectionJava(hospital='JX').data_from_both_connections(
            f"""
                SELECT 
                PatientID,
                Value
                FROM P_DerVals
                WHERE PatientID IN {patlist}
                AND VariableID = 30000350
                AND Value > 0
            """)

        sis.rename({'Value':'sis'}, axis=1, inplace=True)
        
        sis_g = sis.groupby('PatientID', as_index=False).agg({'sis':'last'})
        sis_g['sis_q'] = pd.qcut(sis_g['sis'], 4, labels=False) + 1
        df = pd.merge(df, sis_g, on='PatientID', how='right')
        df.reset_index(drop=True, inplace=True)
        
        del db, patlist, sis, sis_g
        
        print ('hj23 patients >= 16 and ICU LOS > 1 day with SIS: ', df['PatientID'].nunique())
    
    return df

## Patients with lactate measurements at ICU Admission

In [21]:
def lactate(df):
    
    db = df['db'][0]
    
    if db == 'eicu':
        
        patlist = tuple(set(df['patientunitstayid']))
        
        conn = connection_lcp(db)
        
        lactate = pd.read_sql(f"""
                                SELECT 
                                patientunitstayid,
                                labresult AS lactate,
                                labresultoffset AS lactate_offset
                                FROM eicu.lab 
                                WHERE patientunitstayid IN {patlist}
                                AND labresultoffset BETWEEN -360 AND 1440 
                                AND labname = 'lactate'
                                AND labresult BETWEEN 0 AND 50
                                """, conn)
        
        conn.close()

        lactate_clean = lactate.drop_duplicates(['patientunitstayid','lactate'], keep='first')
        lactate_worst = lactate_clean.groupby('patientunitstayid', as_index=False).agg({'lactate':'max'})
        lactate_worst = pd.merge(lactate_worst, lactate_clean, on=['patientunitstayid','lactate'], how='left')
        lactate_worst['lactate_label'] = np.where(lactate_worst['lactate']<2,'normal',
                                         np.where(lactate_worst['lactate']>=4,'high','intermediate'))
        df = pd.merge(df, lactate_worst, on='patientunitstayid', how='right')
        df.reset_index(drop=True, inplace=True)
        
        del db, patlist, lactate_clean, lactate_worst

        print ('eicu patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission: ',
               df['patientunitstayid'].nunique())
        
    elif db == 'mimic':
        
        patlist = tuple(set(df['hadm_id']))

        conn = connection_lcp(db)

        lactate = pd.read_sql(f"""
                                SELECT
                                hadm_id,
                                valuenum AS lactate,
                                charttime AS lactate_time
                                FROM mimiciii.labevents 
                                WHERE hadm_id IN {patlist}
                                AND itemid = 50813
                                AND valuenum BETWEEN 0 AND 50
                                """, conn)

        conn.close()

        lactate_icustayid = pd.merge(lactate, df[['hadm_id','icustay_id','intime']], on='hadm_id', how='left')
        lactate_icustayid['lactate_offset'] = (
            lactate_icustayid['lactate_time'] - lactate_icustayid['intime']).dt.total_seconds()/60
        lactate_icustayid_adm = lactate_icustayid[lactate_icustayid['lactate_offset'].between(-360, 1440)]
        lactate_icustayid_adm_clean = lactate_icustayid_adm[['icustay_id','lactate','lactate_offset']]
        lactate_icustayid_adm_clean.drop_duplicates(['icustay_id','lactate'], keep='first', inplace=True)
        lactate_worst = lactate_icustayid_adm_clean.groupby('icustay_id', as_index=False).agg({'lactate':'max'})
        lactate_worst = pd.merge(lactate_worst, lactate_icustayid_adm_clean, on=['icustay_id','lactate'], how='left')
        lactate_worst['lactate_label'] = np.where(lactate_worst['lactate']<2,'normal',
                                         np.where(lactate_worst['lactate']>=4,'high','intermediate'))
        df = pd.merge(df, lactate_worst, on='icustay_id', how='right')
        df.reset_index(drop=True, inplace=True)
        
        lactate = lactate_icustayid_adm_clean
        
        del db, patlist, lactate_icustayid, lactate_icustayid_adm, lactate_icustayid_adm_clean, lactate_worst

        print ('mimic patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission: ',
               df['icustay_id'].nunique())
        
    else:
        
        patlist = tuple(set(df['PatientID']))

        lactate = ConnectionJava(hospital='JX').data_from_both_connections(
            f"""
                SELECT 
                PatientID,
                Value,
                SampleTime
                FROM P_LabRes
                WHERE PatientID IN {patlist}
                AND VariableID IN (24000658, 24000660)
                AND Value > 0
            """)
        
        lactate.rename({'Value':'lactate','SampleTime':'lactate_time'}, axis=1, inplace=True)
        
        lactate_icustayid = pd.merge(lactate, df[['PatientID','AdmTime']], on='PatientID', how='left')
        lactate_icustayid['lactate_offset'] = (pd.to_datetime(lactate_icustayid['lactate_time']) - pd.to_datetime(
            lactate_icustayid['AdmTime'])).dt.total_seconds()/60
        lactate_icustayid_adm = lactate_icustayid[lactate_icustayid['lactate_offset'].between(-360, 1440)]
        lactate_icustayid_adm_clean = lactate_icustayid_adm[['PatientID','lactate','lactate_offset']]
        lactate_icustayid_adm_clean.drop_duplicates(['PatientID','lactate'], keep='first', inplace=True)
        lactate_worst = lactate_icustayid_adm_clean.groupby('PatientID', as_index=False).agg({'lactate':'max'})
        lactate_worst = pd.merge(lactate_worst, lactate_icustayid_adm_clean, on=['PatientID','lactate'], how='left')
        lactate_worst['lactate_label'] = np.where(lactate_worst['lactate']<2,'normal',
                                            np.where(lactate_worst['lactate']>=4,'high','intermediate'))
        df = pd.merge(df, lactate_worst, on='PatientID', how='right')
        df.reset_index(drop=True, inplace=True)
        
        lactate = lactate_icustayid_adm_clean
        
        del db, patlist, lactate_icustayid, lactate_icustayid_adm, lactate_icustayid_adm_clean, lactate_worst

        print ('hj23 patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission: ',
               df['PatientID'].nunique())
            
    return df, lactate

## Septic patients of the 4th SIS quartile with normal (<2) and high (>=4) lactate records at ICU admission

In [27]:
def septic(df):
    
    db = df['db'][0]
    
    if db == 'eicu':
        
        patlist = tuple(set(df['patientunitstayid']))
        
        conn = connection_lcp(db)
        
        septic = pd.read_sql(f"""
                                SELECT
                                patientunitstayid
                                FROM eicu.patient
                                WHERE patientunitstayid IN {patlist}
                                AND apacheadmissiondx LIKE '%Sepsis%'
                                """, conn)
        
        conn.close()
    
        df_s_allq_il = pd.merge(df, septic, on='patientunitstayid', how='right')
        
        print ("""eicu patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic: """,
                  df_s_allq_il['patientunitstayid'].nunique())
        
        df = df_s_allq_il[df_s_allq_il['sis_q']==4]
        df_s_allq_il.reset_index(drop=True, inplace=True)
        
        print ("""eicu patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic and
                  at 4th SIS quartile: """, df['patientunitstayid'].nunique())
        
        df = df[df['lactate_label']!='intermediate']
        df.reset_index(drop=True, inplace=True)
        
        del db, patlist, septic

        print ("""eicu patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic and
                  at 4th SIS quartile and with normal (<2) or high (>=4) lactate: """,
                  df['patientunitstayid'].nunique())
        
    elif db == 'mimic': 
        
        patlist = tuple(set(df['hadm_id']))
        
        conn = connection_lcp(db)
        
        septic = pd.read_sql(f"""
                                SELECT
                                hadm_id
                                FROM mimiciii.explicit_sepsis
                                WHERE hadm_id IN {patlist}
                                AND sepsis = 1
                                """, conn)
        
        conn.close()
    
        df_s_allq_il = pd.merge(df, septic, on='hadm_id', how='right')
        
        print ("""mimic patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic: """,
                  df_s_allq_il['icustay_id'].nunique())
        
        df = df_s_allq_il[df_s_allq_il['sis_q']==4]
        df_s_allq_il.reset_index(drop=True, inplace=True)
        
        print ("""mimic patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic and
                  at 4th SIS quartile: """, df['icustay_id'].nunique())
        
        df = df[df['lactate_label']!='intermediate']
        df.reset_index(drop=True, inplace=True)
        
        del db, patlist, septic

        print ("""mimic patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic and
                  at 4th SIS quartile and with normal (<2) or high (>=4) lactate: """,
                  df['icustay_id'].nunique())
        
    else:
        
        patlist = tuple(set(df['PatientID']))
        
        sepsis = ConnectionJava(hospital='JX').data_from_both_connections(
            f"""
                SELECT 
                PatientID,
                ReferenceCodeName
                FROM PV_DiagnosesProcedures
                WHERE PatientID IN {patlist}
                AND CodingTypeID IN (6,7,11,12)
            """)
            
        septic = sepsis[(sepsis['ReferenceCodeName'].str.contains('sèpsi|sèpt',case=False)) & (
            ~sepsis['ReferenceCodeName'].str.contains('asèpt',case=False))]
        
        septic.drop('ReferenceCodeName', axis=1, inplace=True)
        septic.drop_duplicates(keep='last', inplace=True)
        
        df_s_allq_il = pd.merge(df, septic, on='PatientID', how='right')
        
        print ("""hj23 patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic: """,
                  df_s_allq_il['PatientID'].nunique())
        
        df = df_s_allq_il[df_s_allq_il['sis_q']==4]
        df_s_allq_il.reset_index(drop=True, inplace=True)
        
        print ("""hj23 patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic and
                  at 4th SIS quartile: """, df['PatientID'].nunique())
        
        df = df[df['lactate_label']!='intermediate']
        df.reset_index(drop=True, inplace=True)
        
        del db, patlist, sepsis, septic

        print ("""hj23 patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic and
                  at 4th SIS quartile and with normal (<2) or high (>=4) lactate: """,
                  df['PatientID'].nunique())

    return df_s_allq_il, df

## Add labs to our filtered cohort

In [22]:
def labs(df):
    
    db = df['db'][0]
    
    if db == 'eicu': # remove alkaline phosp.
        
        patlist = tuple(set(df['patientunitstayid']))
        
        conn = connection_lcp(db)
        
        labs = pd.read_sql(f"""
                            SELECT 
                            *
                            FROM eicu.lab 
                            WHERE patientunitstayid IN {patlist}
                            AND labresultoffset BETWEEN -360 AND 1440 
                            """, conn)
        
        conn.close()
    
        labs_clean = labs[labs['labname'].isin(
            ['glucose','potassium','sodium','chloride','creatinine','BUN','calcium','bicarbonate','platelets x 1000',
             'WBC x 1000','ALT (SGPT)','AST (SGOT)','total bilirubin'])]

        #### Get the nearest measurement to the worst lactate for each lab

        labs_clean = pd.merge(labs_clean, df[['patientunitstayid','lactate_offset']], on='patientunitstayid', how='left')
        labs_clean['lac_distance'] = abs(labs_clean['labresultrevisedoffset'] - labs_clean['lactate_offset'])
        labs_clean_nearest = labs_clean.groupby(['patientunitstayid','labname'], as_index=False).agg({'lac_distance':'min'})
        labs_clean_nearest = pd.merge(labs_clean_nearest,
                                      labs_clean[['patientunitstayid','labname','lac_distance','labresult']],
                                      on = ['patientunitstayid','labname','lac_distance'], how = 'left')
        labs_clean_nearest_unique = labs_clean_nearest.drop_duplicates(['patientunitstayid','labname'], keep='last')
        labs2lac_pivoted = labs_clean_nearest_unique.pivot(index='patientunitstayid', columns='labname', values='labresult')
        labs2lac_pivoted.reset_index(level=0, inplace=True)
        df = pd.merge(df, labs2lac_pivoted, on='patientunitstayid', how='left')
        df = df.rename({'BUN':'bun','platelets x 1000':'platelets','WBC x 1000':'wbc','ALT (SGPT)':'alt',
                        'AST (SGOT)':'ast','total bilirubin':'bilirubin'}, axis=1)
        df.reset_index(drop=True, inplace=True)
        
        del db, patlist, labs, labs_clean, labs_clean_nearest, labs_clean_nearest_unique
        
    elif db == 'mimic': # remove alkaline phosp.
    
        patlist = tuple(set(df['hadm_id']))
        
        conn = connection_lcp(db)
        
        labs = pd.read_sql(f"""
                                SELECT
                                hadm_id,
                                itemid,
                                charttime,
                                valuenum
                                FROM mimiciii.labevents
                                WHERE hadm_id IN {patlist}
                                AND itemid IN (51006,51300,51301,50882,50806,50902,50912,50809,50931,50822,50971,
                                                50824,50983,50885,50861,50878,50893,51265)
                                AND valuenum > 0
                                """, conn)
        
        conn.close()
        
        conn = connection_lcp(db)
        
        labs_names = pd.read_sql(f"""
                                SELECT
                                itemid,
                                label
                                FROM mimiciii.d_labitems
                                WHERE itemid IN (51006,51300,51301,50882,50806,50902,50912,50809,50931,50822,50971,
                                                50824,50983,50885,50861,50878,50893,51265)
                                """, conn)
        
        conn.close()
        
        labs = pd.merge(labs, labs_names, on='itemid', how='left')
        
        labs['labname'] = np.where(labs['label'].str.contains('gluc', case=False), 'glucose',
                        np.where(labs['label'].str.contains('potas', case=False), 'potassium',
                        np.where(labs['label'].str.contains('sodi', case=False), 'sodium',
                        np.where(labs['label'].str.contains('chlor', case=False), 'chloride',
                        np.where(labs['label'].str.contains('creat', case=False), 'creatinine',
                        np.where(labs['label'].str.contains('urea', case=False), 'bun',
                        np.where(labs['label'].str.contains('bicarb', case=False), 'bicarbonate',
                        np.where(labs['label'].str.contains('platel', case=False), 'platelets',
                        np.where(labs['label'].str.contains('calc', case=False), 'calcium',
                        np.where(labs['label'].str.contains('bilir', case=False), 'bilirubin',
                        np.where(labs['label'].str.contains('alanine', case=False), 'alt',
                        np.where(labs['label'].str.contains('asparate', case=False), 'ast',
                                 'wbc'))))))))))))
        
        labs_icustayid = pd.merge(labs, df[['hadm_id','icustay_id','intime']], on='hadm_id', how='left')
        labs_icustayid['labs_offset'] = (
            labs_icustayid['charttime'] - labs_icustayid['intime']).dt.total_seconds()/60
        labs_icustayid_adm = labs_icustayid[labs_icustayid['labs_offset'].between(-360, 1440)]
        
        #### Get the nearest measurement to the worst lactate for each lab
        
        labs2lac = pd.merge(labs_icustayid_adm, df[['icustay_id','lactate_offset']], on='icustay_id', how='left')
        labs2lac['lac_distance'] = abs(labs2lac['labs_offset'] - labs2lac['lactate_offset'])
        labs2lac_nearest = labs2lac.groupby(['icustay_id','labname'], as_index=False).agg({'lac_distance':'min'})
        labs2lac_nearest = pd.merge(labs2lac_nearest,
                                      labs2lac[['icustay_id','labname','lac_distance','valuenum']],
                                      on = ['icustay_id','labname','lac_distance'], how = 'left')
        labs2lac_nearest_unique = labs2lac_nearest.drop_duplicates(['icustay_id','labname'], keep='last')
        labs2lac_pivoted = labs2lac_nearest_unique.pivot(index='icustay_id', columns='labname', values='valuenum')
        labs2lac_pivoted.reset_index(level=0, inplace=True)
        df = pd.merge(df, labs2lac_pivoted, on='icustay_id', how='left')
        df.reset_index(drop=True, inplace=True)
        
        del db, patlist, labs, labs_names, labs_icustayid, labs_icustayid_adm
        del labs2lac, labs2lac_nearest, labs2lac_nearest_unique
        
    else:
        
        patlist = tuple(set(df['PatientID']))

        labsref = ConnectionJava(hospital='JX').chunked_read(
            """
                SELECT DISTINCT 
                VariableID,
                Name
                FROM System..S_VariableRefView 
                WHERE VarType=2
            """, production=True)

        labs = ConnectionJava(hospital='JX').data_from_both_connections(
            f"""
                SELECT 
                PatientID,
                VariableID,
                Value,
                SampleTime
                FROM P_LabRes
                WHERE PatientID IN {patlist}
                AND VariableID IN ( 24001804, 20000400, 24001805, 20000700, 24000589, 24000638,
                24000707, 24000710, 20004200, 20004210, 24000290, 24000434, 24000455, 24000643,
                24001862, 24001976, 24002041, 24002042, 24000711, 20004100, 24000667, 24001806,
                20005100, 24000568, 24000603, 24001803, 24000644, 24000713, 20000600, 24000648,
                24000718, 20005110, 24000725, 24000653, 24000519, 24000579, 24001827, 20000500,
                24000641, 20004300, 24000642, 24002043)
                AND Value > 0
            """)

        labs = pd.merge(labs, labsref, on='VariableID', how='left')

        labs['labname'] = np.where((labs['Name'].str.contains('alani', case=False)) |
                          (labs['Name'].str.contains('ALT', case=True)), 'alt',
                          np.where((labs['Name'].str.contains('aspar', case=False)) |
                          (labs['Name'].str.contains('AST', case=True)), 'ast',
                          np.where(labs['Name'].str.contains('bicarb', case=False), 'bicarbonate',
                          np.where(labs['Name'].str.contains('bilir', case=False), 'bilirubin',
                          np.where(labs['Name'].str.contains('urea', case=False), 'bun',
                          np.where((labs['Name'].str.contains('cal', case=False)) |
                          (labs['Name'].str.contains('ionic', case=False)), 'calcium',
                          np.where(labs['Name'].str.contains('clor', case=False), 'chloride',
                          np.where(labs['Name'].str.contains('creat', case=False), 'creatinine',
                          np.where(labs['Name'].str.contains('gluco', case=False), 'glucose',
                          np.where(labs['Name'].str.contains('plaq', case=False), 'platelets',
                          np.where(labs['Name'].str.contains('sodi', case=False), 'sodium',
                          np.where(labs['Name'].str.contains('leuc', case=False), 'wbc','potassium'))))))))))))
        
        labs['Value'] = np.where(labs['labname']=='bun', labs['Value']/2.1428, labs['Value'])
        
        labs_icustayid = pd.merge(labs, df[['PatientID','AdmTime']], on='PatientID', how='left')
        labs_icustayid['labs_offset'] = (pd.to_datetime(labs_icustayid['SampleTime']) - pd.to_datetime(
            labs_icustayid['AdmTime'])).dt.total_seconds()/60
        labs_icustayid_adm = labs_icustayid[labs_icustayid['labs_offset'].between(-360, 1440)]
        
        #### Get the nearest measurement to the worst lactate for each lab
        
        labs2lac = pd.merge(labs_icustayid_adm, df[['PatientID','lactate_offset']], on='PatientID', how='left')
        labs2lac['lac_distance'] = abs(labs2lac['labs_offset'] - labs2lac['lactate_offset'])
        labs2lac_nearest = labs2lac.groupby(['PatientID','labname'], as_index=False).agg({'lac_distance':'min'})
        labs2lac_nearest = pd.merge(labs2lac_nearest,
                                      labs2lac[['PatientID','labname','lac_distance','Value']],
                                      on = ['PatientID','labname','lac_distance'], how = 'left')
        labs2lac_nearest_unique = labs2lac_nearest.drop_duplicates(['PatientID','labname'], keep='last')
        labs2lac_pivoted = labs2lac_nearest_unique.pivot(index='PatientID', columns='labname', values='Value')
        labs2lac_pivoted.reset_index(level=0, inplace=True)
        df = pd.merge(df, labs2lac_pivoted, on='PatientID', how='left')
        df.reset_index(drop=True, inplace=True)
        
        del db, patlist, labsref, labs, labs_icustayid, labs_icustayid_adm
        del labs2lac, labs2lac_nearest, labs2lac_nearest_unique

    return df, labs2lac_pivoted

## Add features to our filtered cohort

In [23]:
def feats(df): #heart rate, blood pressure and temperature
    
    db = df['db'][0]
    
    if db == 'eicu': # remove alkaline phosp.
        
        patlist = tuple(set(df['patientunitstayid']))
        
        conn = connection_lcp(db)
        
        hr = pd.read_sql(f"""
                            SELECT 
                            patientunitstayid,
                            observationOffset,
                            heartRate AS hr
                            FROM eicu.vitalPeriodic
                            WHERE patientunitstayid IN {patlist}
                            AND heartRate > 0 AND heartRate < 300
                            AND observationOffset BETWEEN -360 AND 1440 
                            """, conn)
        
        bp = pd.read_sql(f"""
                            SELECT 
                            patientunitstayid,
                            observationOffset,
                            systemicMean AS bp
                            FROM eicu.vitalPeriodic
                            WHERE patientunitstayid IN {patlist}
                            AND systemicMean > 0 AND systemicMean < 300
                            AND observationOffset BETWEEN -360 AND 1440 
                            """, conn)
        
        t = pd.read_sql(f"""
                            SELECT 
                            patientunitstayid,
                            observationOffset,
                            temperature
                            FROM eicu.vitalPeriodic
                            WHERE patientunitstayid IN {patlist}
                            AND temperature > 10 AND temperature < 50
                            AND observationOffset BETWEEN -360 AND 1440 
                            """, conn)
        
        conn.close()

        #### Get the nearest measurement to the worst lactate for each feat

        hr = pd.merge(hr, df[['patientunitstayid','lactate_offset']], on='patientunitstayid', how='left')
        bp = pd.merge(bp, df[['patientunitstayid','lactate_offset']], on='patientunitstayid', how='left')
        t = pd.merge(t, df[['patientunitstayid','lactate_offset']], on='patientunitstayid', how='left')
        
        hr['lac_distance'] = abs(hr['observationoffset'] - hr['lactate_offset'])
        bp['lac_distance'] = abs(bp['observationoffset'] - bp['lactate_offset'])
        t['lac_distance'] = abs(t['observationoffset'] - t['lactate_offset'])
        
        hr.drop_duplicates(['patientunitstayid','lac_distance'], keep='first', inplace=True)
        bp.drop_duplicates(['patientunitstayid','lac_distance'], keep='first', inplace=True)
        t.drop_duplicates(['patientunitstayid','lac_distance'], keep='first', inplace=True)
        
        hrG = hr.groupby(['patientunitstayid'], as_index=False).agg({'lac_distance':'min'})
        hrG = pd.merge(hrG, hr, on=['patientunitstayid','lac_distance'], how='left')
        hrG = hrG[['patientunitstayid','hr']]
        
        bpG = bp.groupby(['patientunitstayid'], as_index=False).agg({'lac_distance':'min'})
        bpG = pd.merge(bpG, bp, on=['patientunitstayid','lac_distance'], how='left')
        bpG = bpG[['patientunitstayid','bp']]
        
        tG = t.groupby(['patientunitstayid'], as_index=False).agg({'lac_distance':'min'})
        tG = pd.merge(tG, t, on=['patientunitstayid','lac_distance'], how='left')
        tG = tG[['patientunitstayid','temperature']]
        
        feats_dummy = pd.merge(hrG, bpG, on='patientunitstayid', how='left')
        feats = pd.merge(feats_dummy, tG, on='patientunitstayid', how='left')
        
        del patlist, hr, bp, t, hrG, bpG, tG, feats_dummy
        
        df = pd.merge(df, feats, on='patientunitstayid', how='left')
        
    elif db == 'mimic': # remove alkaline phosp.
    
        patlist = tuple(set(df['icustay_id']))
        
        conn = connection_lcp(db)
        
        hr = pd.read_sql(f"""
                                SELECT
                                icustay_id,
                                charttime,
                                valuenum AS hr
                                FROM mimiciii.chartevents
                                WHERE icustay_id IN {patlist}
                                AND itemid IN (211,220045)
                                AND valuenum > 0 AND valuenum < 300
                                """, conn)
        
        bp = pd.read_sql(f"""
                                SELECT
                                icustay_id,
                                charttime,
                                valuenum AS bp
                                FROM mimiciii.chartevents
                                WHERE icustay_id IN {patlist}
                                AND itemid IN (456,52,6702,443,220052,220181,225312)
                                AND valuenum > 0 AND valuenum < 300
                                """, conn)
        
        t = pd.read_sql(f"""
                                SELECT
                                icustay_id,
                                charttime,
                                CASE 
                                    WHEN itemid IN (223761,678) THEN (valuenum-32)/1.8 ELSE valuenum END AS temperature
                                FROM mimiciii.chartevents
                                WHERE icustay_id IN {patlist}
                                AND itemid IN (223762,676,223761,678)
                                AND valuenum > 10 AND valuenum < 50
                                """, conn)
        
        conn.close()
        
        hr = pd.merge(hr, df[['icustay_id','intime']], on='icustay_id', how='left')
        hr['feat_offset'] = (
            hr['charttime'] - hr['intime']).dt.total_seconds()/60
        hr = hr[hr['feat_offset'].between(-360, 1440)][['icustay_id','feat_offset','hr']]
        
        bp = pd.merge(bp, df[['icustay_id','intime']], on='icustay_id', how='left')
        bp['feat_offset'] = (
            bp['charttime'] - bp['intime']).dt.total_seconds()/60
        bp = bp[bp['feat_offset'].between(-360, 1440)][['icustay_id','feat_offset','bp']]
        
        t = pd.merge(t, df[['icustay_id','intime']], on='icustay_id', how='left')
        t['feat_offset'] = (
            t['charttime'] - t['intime']).dt.total_seconds()/60
        t = t[t['feat_offset'].between(-360, 1440)][['icustay_id','feat_offset','temperature']]
        
        hr = pd.merge(hr, df[['icustay_id','lactate_offset']], on='icustay_id', how='left')
        bp = pd.merge(bp, df[['icustay_id','lactate_offset']], on='icustay_id', how='left')
        t = pd.merge(t, df[['icustay_id','lactate_offset']], on='icustay_id', how='left')
        
        hr['lac_distance'] = abs(hr['feat_offset'] - hr['lactate_offset'])
        bp['lac_distance'] = abs(bp['feat_offset'] - bp['lactate_offset'])
        t['lac_distance'] = abs(t['feat_offset'] - t['lactate_offset'])
        
        hr.drop_duplicates(['icustay_id','lac_distance'], keep='first', inplace=True)
        bp.drop_duplicates(['icustay_id','lac_distance'], keep='first', inplace=True)
        t.drop_duplicates(['icustay_id','lac_distance'], keep='first', inplace=True)
        
        hrG = hr.groupby(['icustay_id'], as_index=False).agg({'lac_distance':'min'})
        hrG = pd.merge(hrG, hr, on=['icustay_id','lac_distance'], how='left')
        hrG = hrG[['icustay_id','hr']]
        
        bpG = bp.groupby(['icustay_id'], as_index=False).agg({'lac_distance':'min'})
        bpG = pd.merge(bpG, bp, on=['icustay_id','lac_distance'], how='left')
        bpG = bpG[['icustay_id','bp']]
        
        tG = t.groupby(['icustay_id'], as_index=False).agg({'lac_distance':'min'})
        tG = pd.merge(tG, t, on=['icustay_id','lac_distance'], how='left')
        tG = tG[['icustay_id','temperature']]
        
        feats_dummy = pd.merge(hrG, bpG, on='icustay_id', how='left')
        feats = pd.merge(feats_dummy, tG, on='icustay_id', how='left')
        
        #del patlist, hr, bp, t, hrG, bpG, tG, feats_dummy
        
        df = pd.merge(df, feats, on='icustay_id', how='left')
        
    else:
        
        patlist = tuple(set(df['PatientID']))
        
        hr = ConnectionJava(hospital='JX').data_from_both_connections(
            f"""
                SELECT
                PatientID,
                DateTime,
                Value
                FROM P_MonVals
                WHERE PatientID IN {patlist}
                AND VariableID = 200
                AND Value > 0 AND Value < 300
            """)
        
        bp = ConnectionJava(hospital='JX').data_from_both_connections(
            f"""
                SELECT
                PatientID,
                DateTime,
                Value
                FROM P_MonVals
                WHERE PatientID IN {patlist}
                AND VariableID IN (110,610)
                AND Value > 0 AND Value < 300
            """)
        
        t = ConnectionJava(hospital='JX').data_from_both_connections(
            f"""
                SELECT
                PatientID,
                DateTime,
                Value
                FROM P_MonVals
                WHERE PatientID IN {patlist}
                AND VariableID = 7102
                AND Value > 10 AND Value < 50
            """)
        
        hr.rename({'Value':'hr'}, axis=1, inplace=True)
        bp.rename({'Value':'bp'}, axis=1, inplace=True)
        t.rename({'Value':'temperature'}, axis=1, inplace=True)
        
        hr = pd.merge(hr, df[['PatientID','AdmTime']], on='PatientID', how='left')
        hr['feat_offset'] = (pd.to_datetime(hr['DateTime']) - pd.to_datetime(hr['AdmTime'])).dt.total_seconds()/60
        hr = hr[hr['feat_offset'].between(-360, 1440)][['PatientID','feat_offset','hr']]
        
        bp = pd.merge(bp, df[['PatientID','AdmTime']], on='PatientID', how='left')
        bp['feat_offset'] = (pd.to_datetime(bp['DateTime']) - pd.to_datetime(bp['AdmTime'])).dt.total_seconds()/60
        bp = bp[bp['feat_offset'].between(-360, 1440)][['PatientID','feat_offset','bp']]
        
        t = pd.merge(t, df[['PatientID','AdmTime']], on='PatientID', how='left')
        t['feat_offset'] = (pd.to_datetime(t['DateTime']) - pd.to_datetime(t['AdmTime'])).dt.total_seconds()/60
        t = t[t['feat_offset'].between(-360, 1440)][['PatientID','feat_offset','temperature']]
        
        hr = pd.merge(hr, df[['PatientID','lactate_offset']], on='PatientID', how='left')
        bp = pd.merge(bp, df[['PatientID','lactate_offset']], on='PatientID', how='left')
        t = pd.merge(t, df[['PatientID','lactate_offset']], on='PatientID', how='left')
        
        hr['lac_distance'] = abs(hr['feat_offset'] - hr['lactate_offset'])
        bp['lac_distance'] = abs(bp['feat_offset'] - bp['lactate_offset'])
        t['lac_distance'] = abs(t['feat_offset'] - t['lactate_offset'])
        
        hr.drop_duplicates(['PatientID','lac_distance'], keep='first', inplace=True)
        bp.drop_duplicates(['PatientID','lac_distance'], keep='first', inplace=True)
        t.drop_duplicates(['PatientID','lac_distance'], keep='first', inplace=True)
        
        hrG = hr.groupby(['PatientID'], as_index=False).agg({'lac_distance':'min'})
        hrG = pd.merge(hrG, hr, on=['PatientID','lac_distance'], how='left')
        hrG = hrG[['PatientID','hr']]
        
        bpG = bp.groupby(['PatientID'], as_index=False).agg({'lac_distance':'min'})
        bpG = pd.merge(bpG, bp, on=['PatientID','lac_distance'], how='left')
        bpG = bpG[['PatientID','bp']]
        
        tG = t.groupby(['PatientID'], as_index=False).agg({'lac_distance':'min'})
        tG = pd.merge(tG, t, on=['PatientID','lac_distance'], how='left')
        tG = tG[['PatientID','temperature']]
        
        feats_dummy = pd.merge(hrG, bpG, on='PatientID', how='left')
        feats = pd.merge(feats_dummy, tG, on='PatientID', how='left')
        
        del patlist, hr, bp, t, hrG, bpG, tG, feats_dummy
        
        df = pd.merge(df, feats, on='PatientID', how='left')

    return df, feats

## Add comorbidities

In [13]:
def mimicsql_comorbidities_template4eicu(cohort):

    patlist = tuple(set(cohort['patientunitstayid']))
    
    conn = connection_lcp('eicu')
        
    diags = pd.read_sql(f""" 
                        SELECT
                        patientunitstayid,
                        CASE 
                            WHEN (substring(icd9code,1,6) = ('398.91')) THEN 1
                            WHEN (substring(icd9code,1,3) = ('428')) THEN 1
                        END AS CHF,
                        CASE 
                            WHEN substring(ICD9CODE,1,6) = '426.10' then 1
                            WHEN substring(ICD9CODE,1,6) = '426.11' then 1
                            WHEN substring(ICD9CODE,1,6) = '426.13' then 1
                            WHEN substring(ICD9CODE,1,5) IN ('426.2','426.3','426.4') then 1
                            WHEN substring(ICD9CODE,1,6) IN ('426.50','426.51','426.52','426.53') then 1
                            WHEN substring(ICD9CODE,1,5) IN ('426.6','426.7') then 1
                            WHEN substring(ICD9CODE,1,6) IN ('426.80','426.81','426.82','426.83','426.84','426.85','426.86','426.87','426.88','426.89') then 1
                            WHEN substring(ICD9CODE,1,5) IN ('427.0') then 1
                            WHEN substring(ICD9CODE,1,5) IN ('427.2') then 1
                            WHEN substring(ICD9CODE,1,6) IN ('427.31') then 1
                            WHEN substring(ICD9CODE,1,6) IN ('427.60') then 1
                            WHEN substring(ICD9CODE,1,5) IN ('427.9') then 1
                            WHEN substring(ICD9CODE,1,5) IN ('785.0') then 1
                            WHEN substring(ICD9CODE,1,5) IN ('V45.0') then 1
                            WHEN substring(ICD9CODE,1,6) IN ('V45.00','V45.01','V45.02','V45.03','V45.04','V45.05','V45.06','V45.07','V45.08','V45.09') then 1
                            WHEN substring(ICD9CODE,1,5) IN ('V53.3') then 1
                            WHEN substring(ICD9CODE,1,6) IN ('V53.30','V53.31','V53.32','V53.33','V53.34','V53.35','V53.36','V53.37','V53.38','V53.39' ) then 1
                        END AS ARYTHM,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('093.20','093.21','093.22','093.23','093.24') then 1
                            when substring(ICD9CODE,1,3) IN ('394', '395','396') then 1
                            when substring(ICD9CODE,1,5) IN ('397.0','397.1') then 1
                            when substring(ICD9CODE,1,5) IN ('397.9') then 1
                            when substring(ICD9CODE,1,5) IN ('424.0', '424.1','424.2', '424.3','424.4', '424.5','424.6', '424.7','424.8', '424.9')  then 1
                            when substring(ICD9CODE,1,5) IN ('746.3', '746.4','746.5','746.6') then 1
                            when substring(ICD9CODE,1,5) IN ('V42.2')then 1
                            when substring(ICD9CODE,1,5) IN ('V43.3') then 1
                        end as VALVE,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('415.11','415.12','415.13','415.14','415.15','415.16','415.17','415.18','415.19') then 1
                            when substring(ICD9CODE,1,5) IN ('416.0','416.1','416.2','416.3','416.4','416.5','416.6','416.7','416.8','416.9') then 1
                            when substring(ICD9CODE,1,5) IN ('417.9') then 1
                        end as PULMCIRC,
                        CASE
                            when substring(ICD9CODE,1,5) IN ('440.0','440.1','440.2','440.3','440.4','440.5','440.6','440.7','440.8','440.9') then 1
                            when substring(ICD9CODE,1,5) IN ('441.0','441.1','441.2','441.3','441.4','441.5','441.6','441.7','441.8','441.9') then 1
                            when substring(ICD9CODE,1,5) IN ('442.0','442.1','442.2','442.3','442.4','442.5','442.6','442.7','442.8','442.9')then 1
                            when substring(ICD9CODE,1,5) IN ('443.0','443.1','443.2','443.3','443.4','443.5','443.6','443.7','443.8','443.9') then 1
                            when substring(ICD9CODE,1,6) IN ('444.21','444.22') then 1
                            when substring(ICD9CODE,1,5) IN ('447.1') then 1
                            when substring(ICD9CODE,1,3) IN ('449') then 1
                            when substring(ICD9CODE,1,5) IN ('557.1','557.9','V43.4') then 1
                        end as PERIVASC,
                        CASE
                            when substring(ICD9CODE,1,5) IN ('401.1','401.9') then 1
                            when substring(ICD9CODE,1,6) IN ('642.00','642.04','642.01','642.02','642.03') then 1
                        end as HTN,
                        CASE
                            when substring(ICD9CODE,1,5) IN ('401.0','437.2') then 1
                        end as HTNCX,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('642.20','642.21','642.22','642.23','642.24') then 1
                        end as HTNPREG,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('402.00','402.10','402.90','405.09','405.19','405.99') then 1
                        end as HTNWOCHF,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('402.01','402.11','402.91') then 1
                        end as HTNWCHF,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('403.00','403.10','403.90','405.01','405.11','405.91','642.10','642.11','642.12','642.13','642.14') then 1
                        end as HRENWORF,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('403.01','403.11','403.91') then 1
                        end as HRENWRF,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('404.00','404.10','404.90') then 1
                        end as HHRWOHRF,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('404.01','404.11','404.91') then 1
                        end as HHRWCHF,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('404.02','404.12','404.92') then 1
                        end as HHRWRF,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('404.03','404.13','404.93') then 1
                        end as HHRWHRF,
                        CASE
                            when substring(ICD9CODE,1,6) IN ('642.70','642.71','642.72','642.73','642.74','642.90','642.91','642.92','642.93','642.94') then 1
                        end as OHTNPREG,
                        CASE
                            when substring(ICD9CODE,1,3) IN ('342','343','344') then 1
                            when substring(ICD9CODE,1,5) IN ('438.2','438.3','438.4') then 1
                            when substring(ICD9CODE,1,6) IN ('438.50','438.51','438.52','438.53','780.72') then 1
                        end as PARA,
                        CASE
                            when substring(ICD9CODE,1,3) IN ('330','331') then 1
                            when substring(ICD9CODE,1,5) = '332.0' then 1
                            when substring(ICD9CODE,1,5) = '333.4' then 1
                            when substring(ICD9CODE,1,5) = '333.5' then 1
                            when substring(ICD9CODE,1,5) = '333.7' then 1
                            when substring(ICD9CODE,1,6) in ('333.71','333.72','333.79','333.85','333.94') then 1
                            when substring(ICD9CODE,1,5) in ('334.0','334.1','334.2','334.3','334.4','334.5','334.6','334.7','334.8','334.9','335.0','335.1','335.2','335.3','335.4','335.5','335.6','335.7','335.8') then 1
                            when substring(ICD9CODE,1,5) = ('335.9') then 1
                            when substring(ICD9CODE,1,5) = '338.0' then 1
                            when substring(ICD9CODE,1,3) = '340' then 1
                            when substring(ICD9CODE,1,5) in ('341.1','341.2','341.3','341.4','341.5','341.6') then 1
                            when substring(ICD9CODE,1,6) in ('345.00','345.01','345.02','345.03','345.04','345.05','345.06','345.07','345.08','345.09','345.11','345.10') then 1
                            when substring(ICD9CODE,1,6) in ('345.20','345.21','345.22','345.23','345.24','345.25','345.26','345.27','345.28','345.29','345.30') then 1
                            when substring(ICD9CODE,1,5) in ('345.4','345.5','345.6','345.7','345.8') then 1
                            when substring(ICD9CODE,1,6) in ('345.90','345.91') then 1
                            when substring(ICD9CODE,1,6) in ('347.00','347.01') then 1
                            when substring(ICD9CODE,1,6) in ('347.10','347.11') then 1
                            when substring(ICD9CODE,1,5) = ('348.3') then 1 -- discontinued icd-9
                            when substring(ICD9CODE,1,6) in ('649.40','649.41','649.42','649.43','649.44') then 1
                            when substring(ICD9CODE,1,5) = ('768.7') then 1
                            when substring(ICD9CODE,1,6) in ('768.70','768.71','768.72','768.73') then 1
                            when substring(ICD9CODE,1,5) = ('780.3') then 1
                            when substring(ICD9CODE,1,6) in ('780.31','780.32','780.33','780.39','780.97') then 1
                            when substring(ICD9CODE,1,5) = ('784.3') then 1
                        end as NEURO,
                        CASE
                            when substring(ICD9CODE,1,3) in ('490','491') then 1
                            when substring(ICD9CODE,1,5) in ('492.0','492.1','492.2','492.3','492.4','492.5','492.6','492.7','492.8') then 1
                            when substring(ICD9CODE,1,5) in ('493.0','493.1','493.2','493.3','493.4','493.5','493.6','493.7','493.8') then 1    
                            when substring(ICD9CODE,1,5) in ('493.92','493.91','493.90') then 1
                            when substring(ICD9CODE,1,5) in ('494.0','494.1') then 1
                            when substring(ICD9CODE,1,3) in ('495','496','497','498','499','500','501','502','503','504','505') then 1
                            when substring(ICD9CODE,1,5) = ('506.4') then 1
                        end as CHRNLUNG,
                        CASE
                            when substring(ICD9CODE,1,5) in ('250.0','250.1','250.2') then 1
                            when substring(ICD9CODE,1,6) in ('250.30','250.31','250.32','250.33') then 1
                            when substring(ICD9CODE,1,6) in ('648.00','648.01','648.02','648.03','648.04') then 1
                            when substring(ICD9CODE,1,5) in ('249.0','249.1','249.2') then 1
                            when substring(ICD9CODE,1,6) in ('249.30','249.31') then 1
                        end as DM,
                        CASE
                            when substring(ICD9CODE,1,5) in ('250.4','250.5','250.6','250.7','250.8') then 1
                            when substring(ICD9CODE,1,6) in ('250.90','250.91','250.92','250.93') then 1
                            when substring(ICD9CODE,1,5) = ('775.1') then 1
                            when substring(ICD9CODE,1,5) in ('249.4','249.5','249.6','249.7','249.8') then 1
                            when substring(ICD9CODE,1,6) in ('249.90','249.91') then 1
                        end as DMCX,
                        CASE
                            when substring(ICD9CODE,1,3) in ('243') then 1
                            when substring(ICD9CODE,1,5) in ('244.0','244.1','244.2') then 1
                            when substring(ICD9CODE,1,5) = ('244.8') then 1
                            when substring(ICD9CODE,1,5) = ('244.9') then 1
                        end as HYPOTHY,
                        CASE
                            when substring(ICD9CODE,1,5) =('585.3') then 1
                            when substring(ICD9CODE,1,5) =('585.4') then 1
                            when substring(ICD9CODE,1,5) =('585.5') then 1
                            when substring(ICD9CODE,1,5) =('585.6') then 1
                            when substring(ICD9CODE,1,3) =('586') then 1
                            when substring(ICD9CODE,1,5) =('V42.0') then 1
                            when substring(ICD9CODE,1,5) =('V45.1') then 1
                            when substring(ICD9CODE,1,5) in('V56.0','V56.1','V56.2') then 1
                            when substring(ICD9CODE,1,6) in('V56.30','V56.31','V56.32') then 1
                            when substring(ICD9CODE,1,5) =('V56.8') then 1
                            when substring(ICD9CODE,1,6) in ('V45.11','V45.12') then 1
                        end as RENLFAIL,
                        CASE
                            when substring(ICD9CODE,1,5) = ('456.0') then 1
                            when substring(ICD9CODE,1,5) = ('456.1') then 1
                            when substring(ICD9CODE,1,6) = ('456.20') then 1
                            when substring(ICD9CODE,1,6) = ('456.21') then 1
                            when substring(ICD9CODE,1,5) = ('571.0') then 1
                            when substring(ICD9CODE,1,5) = ('571.3') then 1
                            when substring(ICD9CODE,1,5) = ('571.8') then 1
                            when substring(ICD9CODE,1,5) = ('571.9') then 1
                            when substring(ICD9CODE,1,5) = ('572.3') then 1
                            when substring(ICD9CODE,1,5) = ('572.8') then 1
                            when substring(ICD9CODE,1,5) = ('573.5') then 1
                            when substring(ICD9CODE,1,5) = ('V42.7') then 1
                            when substring(ICD9CODE,1,6) = ('070.22') then 1
                            when substring(ICD9CODE,1,6) = ('070.23') then 1
                            when substring(ICD9CODE,1,6) = ('070.32') then 1
                            when substring(ICD9CODE,1,6) = ('070.33') then 1
                            when substring(ICD9CODE,1,6) = ('070.44') then 1
                            when substring(ICD9CODE,1,6) = ('070.54') then 1
                            when substring(ICD9CODE,1,6) IN ('571.4') then 1
                            when substring(ICD9CODE,1,5) = ('571.2') then 1
                            when substring(ICD9CODE,1,5) = ('571.5') then 1
                            when substring(ICD9CODE,1,5) = ('571.6') then 1
                        end as LIVER,
                        CASE
                            when substring(ICD9CODE,1,3) IN ('200','201','203') then 1
                            when substring(ICD9CODE,1,5) IN ('202.0','202.1','202.2','202.3','202.4') then 1
                            when substring(ICD9CODE,1,5) IN ('202.5','202.6','202.7','202.8','202.9') then 1
                            when substring(ICD9CODE,1,5)= ('238.6') then 1
                            when substring(ICD9CODE,1,5)=('273.3') then 1
                            when substring(ICD9CODE,1,3) IN ('204','205','206','207','208') then 1
                        end as LYMPH,
                        CASE
                            when substring(ICD9CODE,1,3) IN ('196', '197', '198') then 1
                            when substring(ICD9CODE,1,5) IN ('199.0','199.1') then 1
                            when substring(ICD9CODE,1,6) IN ('209.70','209.71','209.72','209.73','209.74','209.75','209.79','789.51' ) then 1
                        end as METS,
                        CASE
                            when substring(ICD9CODE,1,3) IN ('140','141','142','143','144','145','146','147','148','149','150','151','152','153','154','155','156','157','158','159','160','161','162','163','164','165','166','167','168','169','170','171','172') THEN 1
                            when substring(ICD9CODE,1,3) IN ('174','175') then 1
                            when substring(ICD9CODE,1,3) IN ('179','180','181','182','183','184','185','186','187','188','189','190','191','192','193','194') then 1
                            when substring(ICD9CODE,1,5) IN ('195.0','195.1','195.2','195.3','195.4','195.5','195.6','195.7','195.8') then 1
                            when substring(ICD9CODE,1,5) IN ('209.0','209.1','209.2','209.3') THEN 1
                            when substring(ICD9CODE,1,6) IN ('258.01','258.02','258.03') then 1
                        end as TUMOR,
                        CASE
                            when substring(ICD9CODE,1,5) = ('701.0') then 1
                            when substring(ICD9CODE,1,3) = ('710') then 1
                            when substring(ICD9CODE,1,5) IN ('714.0','714.1','714.2','714.3','714.4','714.4','714.5','714.6','714.7','714.8','714.9') then 1
                            when substring(ICD9CODE,1,5) IN ('720.0','720.1','720.2','720.3','720.4','720.4','720.5','720.6','720.7','720.8','720.9') then 1
                            when substring(ICD9CODE,1,3) = ('725') then 1
                        end as ARTH,
                        CASE
                            when substring(ICD9CODE,1,5) IN ('286.0','286.1','286.2','286.3','286.4','286.4','286.5','286.6','286.7','286.8','286.9') then 1
                            when substring(ICD9CODE,1,5) = ('287.1') then 1
                            when substring(ICD9CODE,1,5) IN ('287.3', '287.5','287.4') then 1
                            when substring(ICD9CODE,1,6) IN ('649.30','649.34','649.31','649.32','649.33','289.84') then 1
                        end as COAG,
                        CASE
                            when substring(ICD9CODE,1,5) = ('278.0') then 1
                            when substring(ICD9CODE,1,6) = ('278.00') then 1
                            when substring(ICD9CODE,1,6) = ('278.01') then 1
                            when substring(ICD9CODE,1,6) = ('278.03') then 1
                            when substring(ICD9CODE,1,6) IN ('649.10','649.11','649.12','649.13','649.14') then 1
                            when substring(ICD9CODE,1,5) IN ('V85.3') then 1
                            when substring(ICD9CODE,1,5) IN ('V85.4') then 1
                            when substring(ICD9CODE,1,6) IN ('V85.41','V85.42','V85.43','V85.44','V85.45') then 1
                            when substring(ICD9CODE,1,6) IN ('V85.54') then 1
                            when substring(ICD9CODE,1,6) IN ('793.91') then 1
                        end as OBESE,
                        CASE
                            when substring(ICD9CODE,1,5) IN ('276.0','276.1','276.2','276.3','276.4','276.5','276.6','276.7','276.8','276.9') then 1
                        end as LYTES,
                        CASE
                            when substring(ICD9CODE,1,5) IN ('280.1','280.2','280.2','280.3','280.4','280.5','280.6','280.7','280.8','280.9','281.0','281.1','281.2','281.3','281.4','281.5','281.6','281.7','281.8','281.9') then 1
                            when substring(ICD9CODE,1,6) IN ('285.21','285.22','285.23','285.24','285.25','285.26','285.27','285.28','285.29') THEN 1
                            when substring(ICD9CODE,1,5) = ('285.9') then 1
                        end as ANEMDEF,
                        CASE
                            when substring(ICD9CODE,1,5) IN ('291.0','291.3','291.1','291.2','291.5') then 1
                            when substring(ICD9CODE,1,6) = ('291.8,') then 1
                            when substring(ICD9CODE,1,6) IN ('291.81','291.82','291.89') then 1
                            when substring(ICD9CODE,1,5) = ('291.9') then 1
                            when substring(ICD9CODE,1,6) IN ('303.0','303.1','303.2','303.3','303.4','303.5','303.6','303.7','303.8') THEN 1 
                            when substring(ICD9CODE,1,6) IN ('303.93','303.90','303.91','303.92') then 1
                            when substring(ICD9CODE,1,6) IN ('305.00','305.01','305.02','305.03') then 1
                        end as ALCOHOL,
                        CASE
                            when substring(ICD9CODE,1,5) = ('292.0') then 1
                            when substring(ICD9CODE,1,6) IN ('292.82','292.83','292.84','292.85','292.86','292.87','292.88','292.89') then 1
                            when substring(ICD9CODE,1,5) = ('292.9') then 1
                            when substring(ICD9CODE,1,5) IN ('304.0','304.1','304.2','304.3','304.4','304.5','304.6','304.7','304.8') then 1
                            when substring(ICD9CODE,1,6) IN ('304.93','304.90','304.91','304.92') then 1
                            when substring(ICD9CODE,1,5) IN ('305.2','305.3','305.4','305.5','305.6','305.7','305.8') then 1
                            when substring(ICD9CODE,1,6) IN ('305.93','305.90','305.91','305.92') then 1
                            when substring(ICD9CODE,1,6) IN ('648.30','648.31','648.32','648.33') then 1
                        end as DRUG,
                        CASE
                            when substring(ICD9CODE,1,3) IN ('295','296','297') then 1
                            when substring(ICD9CODE,1,5) IN ('298.0','298.1','298.2','298.3','298.4','298.5','298.6','298.7','298.8','298.9') then 1
                            when substring(ICD9CODE,1,6) = ('299.10') then 1
                            when substring(ICD9CODE,1,6) = ('299.11') then 1
                        end as PSYCH,
                        CASE
                            when substring(ICD9CODE,1,5) IN ('300.4','309.0','309.1') then 1
                            when substring(ICD9CODE,1,6) = ('301.12') then 1
                            when substring(ICD9CODE,1,3) = ('311')        then 1
                        end as DEPRESS
                        FROM eicu.diagnosis 
                        WHERE patientunitstayid IN {patlist}
                        AND diagnosisPriority != 'Primary' """,
                    conn)
        
    conn.close()
    
    diags.fillna(0, inplace=True)
    comorb_dummy = diags.groupby('patientunitstayid', as_index=False)
    df = comorb_dummy.max()
    df.reset_index(drop='True', inplace=True)
    
    del diags, comorb_dummy
    
    #CONGESTIVE_HEART_FAILURE
    df.loc[df.chf==1,'congestive_heart_failure'] =1
    df.loc[df.htnwchf==1, 'congestive_heart_failure'] =1
    df.loc[df.hhrwchf==1, 'congestive_heart_failure'] =1
    df.loc[df.hhrwhrf==1, 'congestive_heart_failure'] =1
    #CARDIAC_ARRHYTHMIAS
    df.loc[df.arythm==1,'cardiac_arrhythmias'] =1
    #VALVULAR_DISEASE
    df.loc[df.valve==1,'valvular_disease'] =1
    #PULMONARY_CIRCULATION
    df.loc[df.pulmcirc==1,'pulmonary_circulation'] =1
    #PERIPHERAL_VASCULAR
    df.loc[df.perivasc==1,'peripheral_vascular'] =1
    #HYPERTENSION
    df.loc[df.htn==1,'hypertension'] =1
    df.loc[df.htncx==1, 'hypertension'] =1
    df.loc[df.htnpreg==1, 'hypertension'] =1
    df.loc[df.htnwochf==1, 'hypertension'] =1
    df.loc[df.htnwchf==1, 'hypertension'] =1
    df.loc[df.hrenworf==1, 'hypertension'] =1
    df.loc[df.hrenwrf==1, 'hypertension'] =1
    df.loc[df.hhrwohrf==1, 'hypertension'] =1
    df.loc[df.hhrwchf==1, 'hypertension'] =1
    df.loc[df.hhrwrf==1, 'hypertension'] =1
    df.loc[df.hhrwhrf==1, 'hypertension'] =1
    df.loc[df.ohtnpreg==1, 'hypertension'] =1
    #PARALYSIS
    df.loc[df.para==1,'paralysis'] =1
    #OTHER_NEUROLOGICAL
    df.loc[df.neuro==1,'other_neurological'] =1
    #CHRONIC_PULMONARY
    df.loc[df.chrnlung==1,'chronic_pulmonary'] =1
    #HYPOTHYROIDISM
    df.loc[df.hypothy==1,'hypothyroidism'] =1
    #DIABETES
    df.loc[df.dm==1,'diabetes'] =1
    df.loc[df.dmcx==1,'diabetes'] =1
    #RENAL_FAILURE
    df.loc[df.renlfail==1,'renal_failure'] =1
    df.loc[df.hrenwrf==1, 'renal_failure'] =1
    df.loc[df.hhrwrf==1, 'renal_failure'] =1
    df.loc[df.hhrwhrf==1, 'renal_failure'] =1
    #LIVER_DISEASE
    df.loc[df.liver==1,'liver_disease'] =1
    #LYMPHOMA
    df.loc[df.lymph==1,'lymphoma'] =1
    #METASTATIC_CANCER
    df.loc[df.mets==1,'metastatic_cancer'] =1
    #SOLID_TUMOR
    df.loc[df.mets==1,'solid_tumor'] =1
    df.loc[df.tumor==1,'solid_tumor'] =1
    #RHEUMATOID_ARTHRITIS
    df.loc[df.arth==1,'rheumatoid_arthritis'] =1
    #COAGULOPATHY
    df.loc[df.coag==1,'coagulopathy'] =1
    #OBESITY
    df.loc[df.obese==1,'obesity'] =1
    #FLUID_ELECTROLYTE
    df.loc[df.lytes==1,'fluid_electrolyte'] =1
    #DEFICIENCY_ANEMIAS
    df.loc[df.anemdef==1,'deficiency_anemias'] =1
    #ALCOHOL_ABUSE
    df.loc[df.alcohol==1,'alcohol_abuse'] =1
    #DRUG_ABUSE
    df.loc[df.drug==1,'drug_abuse'] =1
    #PSYCHOSES
    df.loc[df.psych==1,'psychoses'] =1
    #DEPRESSION
    df.loc[df.depress==1,'depression'] =1
    ##order_all

    df = df.drop(columns=['chf','arythm','valve','pulmcirc','perivasc','htn','htncx','htnpreg','htnwochf',
                          'htnwchf','hrenworf','hrenwrf','hhrwohrf','hhrwchf','hhrwrf','hhrwhrf','ohtnpreg',
                          'para','neuro','chrnlung','dm','dmcx','hypothy','renlfail','liver',
                          'lymph','mets','tumor','arth','coag','obese','lytes','anemdef',
                          'alcohol','drug','psych','depress'])

    
    df.fillna(0, inplace=True)
    df = df.astype(int)
    
    return df


In [14]:
def pasthistory_comorbidities4eicu(cohort):
    
    patlist = tuple(set(cohort['patientunitstayid']))

    conn = connection_lcp('eicu')
        
    pasthist = pd.read_sql(f"""
                        SELECT 
                        patientunitstayid,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF - class I',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF - class II',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF - severity unknown',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF - class III',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF - class IV') 
                        THEN 1 ELSE 0 END AS CONGESTIVE_HEART_FAILURE,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Arrhythmias/atrial fibrillation - chronic',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Arrhythmias/atrial fibrillation - intermittent',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Arrhythmias/MAT',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Arrhythmias/sick sinus syndrome',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Arrhythmias/SVT- other',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Arrhythmias/ventricular ectopy',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Arrhythmias/ventricular fibrillation',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Arrhythmias/ventricular tachycardia')
                        THEN 1 ELSE 0 END AS CARDIAC_ARRHYTHMIAS,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Valve disease/AR',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Valve disease/AS',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Valve disease/MR',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Valve disease/MS',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Valve disease/PS',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Valve disease/s/p AVR',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Valve disease/s/p MVR',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Valve disease/s/p TVR',
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Valve disease/TR')
                        THEN 1 ELSE 0 END AS VALVULAR_DISEASE,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Peripheral Vascular Disease/peripheral vascular disease' )
                        THEN 1 ELSE 0 END AS PERIPHERAL_VASCULAR,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Hypertension Requiring Treatment/hypertension requiring treatment')
                        THEN 1 ELSE 0 END AS HYPERTENSION,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Pulmonary/COPD/COPD  - no limitations',
    'notes/Progress Notes/Past History/Organ Systems/Pulmonary/COPD/COPD  - moderate',
    'notes/Progress Notes/Past History/Organ Systems/Pulmonary/COPD/COPD  - severe')
                        THEN 1 ELSE 0 END AS CHRONIC_PULMONARY,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Endocrine (R)/Hypothyroidism/hypothyroidism')
                        THEN 1 ELSE 0 END AS HYPOTHYROIDISM,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Endocrine (R)/Insulin Dependent Diabetes/insulin dependent diabetes',
    'notes/Progress Notes/Past History/Organ Systems/Endocrine (R)/Non-Insulin Dependent Diabetes/non-medication dependent',
    'notes/Progress Notes/Past History/Organ Systems/Endocrine (R)/Non-Insulin Dependent Diabetes/medication dependent')
                        THEN 1 ELSE 0 END AS DIABETES,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Insufficiency/renal insufficiency - creatinine 3-4',
    'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Insufficiency/renal insufficiency - creatinine > 5',
    'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Insufficiency/renal insufficiency - creatinine 4-5',
    'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Failure/renal failure - peritoneal dialysis',
    'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Failure/renal failure- not currently dialyzed',
    'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Failure/renal failure - hemodialysis')
                        THEN 1 ELSE 0 END AS RENAL_FAILURE,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/clinical diagnosis',
    'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/biopsy proven',
    'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/UGI bleeding',
    'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/varices',
    'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/coma',
    'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/jaundice',
    'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/ascites',
    'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/encephalopathy',
    'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/s/p Liver Transplant/s/p liver transplant')
                        THEN 1 ELSE 0 END AS LIVER_DISEASE,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/non-Hodgkins lymphoma',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/Hodgkins disease',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/multiple myeloma',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/CLL',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/CML',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/leukemia - other',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/other hematologic malignancy',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/AML',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/ALL')
                        THEN 1 ELSE 0 END AS LYMPHOMA,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/other',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/brain',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/carcinomatosis',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/nodes',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/lung',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/intra-abdominal',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/bone',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/liver')
                        THEN 1 ELSE 0 END AS METASTATIC_CANCER,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/bone',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/stomach',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/bile duct',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/kidney',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/unknown',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/breast',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/uterus',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/prostate',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/liver',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/pancreas - adenocarcinoma',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/ovary',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/sarcoma',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/other',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/testes',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/lung',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/melanoma',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/head and neck',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/esophagus',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/bladder',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/none',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/pancreas - islet cell',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/colon',
    'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/brain')
                        THEN 1 ELSE 0 END AS SOLID_TUMOR,
                        CASE WHEN pasthistorypath IN (
    'notes/Progress Notes/Past History/Organ Systems/Rheumatic/SLE/SLE',
    'notes/Progress Notes/Past History/Organ Systems/Rheumatic/Rheumatoid Arthritis/rheumatoid arthritis',
    'notes/Progress Notes/Past History/Organ Systems/Rheumatic/Scleroderma/scleroderma',
    'notes/Progress Notes/Past History/Organ Systems/Rheumatic/Vasculitis/vasculitis',
    'notes/Progress Notes/Past History/Organ Systems/Rheumatic/Dermato/Polymyositis/dermatomyositis')
                        THEN 1 ELSE 0 END AS RHEUMATOID_ARTHRITIS
                        FROM eicu.pasthistory 
                        WHERE patientunitstayid IN {patlist}
                            """, conn)
        
    conn.close()
    
    comorb_dummy = pasthist.groupby('patientunitstayid', as_index=False)
    df = comorb_dummy.max()
    df.reset_index(drop='True', inplace=True)
    
    return df
    

In [15]:
def diag_parser(df):
    
    i10=pd.read_csv("2018_I10gem.txt", header=None, delim_whitespace=True)
    i10.columns = ['i10','i9','extra']
    
    patlist = tuple(set(df['PatientID']))
    
    # 1st we need to extract all secondary diagnoses from our database, where we have icd9 and ic10 codes mixed

    secdiags = ConnectionJava(hospital='JX').data_from_both_connections(
        f"""
            SELECT 
            PatientID,
            ReferenceCode
            FROM PV_DiagnosesProcedures
            WHERE PatientID IN {patlist}
            AND CodingTypeID = 7
        """)
    
    secdiags.drop_duplicates(keep='last', inplace=True)
    
    # 2nd we need to parse the codes to ensure matching with MIMIC sql comorbidities assignments
    
    secdiags['ReferenceCodeParsed'] = secdiags['ReferenceCode'].str.replace(r'.', '')
    secdiags['ReferenceCodeParsed'] = secdiags['ReferenceCodeParsed'].str.split(r'\/').str.get(0)
    
    # 3th we need to add the icd9 codes for those diagnoses we have in icd10 in our database using the icd9_icd10_parser
    # Note that we need to find for the raw code in our database (comorb1), the raw code adding a 0 (comorb2)
    # and the raw code adding 00 (comorb3) to ensure matchings
    
    comorb1 = pd.merge(secdiags[['PatientID','ReferenceCodeParsed']], i10,
                   right_on=['i10'], left_on='ReferenceCodeParsed', how='left')
    comorb2 = pd.merge(secdiags[['PatientID','ReferenceCodeParsed']], i10,
                   right_on=['i10'], left_on=secdiags.ReferenceCodeParsed+'0', how='left')
    comorb3 = pd.merge(secdiags[['PatientID','ReferenceCodeParsed']], i10,
                   right_on=['i10'], left_on=secdiags.ReferenceCodeParsed+'00', how='left')
    
    comorb1notna = comorb1.loc[comorb1['i9'].notna()]
    comorb2notna = comorb2.loc[comorb2['i9'].notna()]
    comorb3notna = comorb3.loc[comorb3['i9'].notna()]
    comorbsnotna = pd.concat([comorb1notna,comorb2notna,comorb3notna])
    
    del comorb1, comorb2, comorb3, comorb1notna, comorb2notna, comorb3notna
    
    # 4th we need to concat our initial dataframe with icd9 and icd10 with the parsed ones to have all icd9 codes
    # which are needed to assign comorbidities using the MIMIC sql template
    
    secdiags_clean = secdiags[['PatientID','ReferenceCodeParsed']]
    secdiags_clean.rename({'ReferenceCodeParsed':'i9'}, axis=1, inplace=True)
    comorbsnotna_clean = comorbsnotna[['PatientID','i9']]
    diags2matchcomorb = pd.concat([secdiags_clean, comorbsnotna_clean])
    
    del secdiags, comorbsnotna, secdiags_clean, comorbsnotna_clean
    
    return diags2matchcomorb

In [16]:
def mimicsql_comorbidities_template4hj23(df):

    ##CHF
    df.loc[df.i9=='39891','CHF'] =1
    df.loc[df.i9.between('4280','4289'),'CHF'] =1
    ##ARYTHM
    df.loc[df.i9=='42610','ARYTHM'] =1
    df.loc[df.i9=='42611','ARYTHM'] =1
    df.loc[df.i9=='42613','ARYTHM'] =1
    df.loc[df.i9=='4270','ARYTHM'] =1
    df.loc[df.i9=='4272','ARYTHM'] =1
    df.loc[df.i9=='42731','ARYTHM'] =1
    df.loc[df.i9=='42760','ARYTHM'] =1
    df.loc[df.i9=='4279','ARYTHM'] =1
    df.loc[df.i9=='7850','ARYTHM'] =1
    df.loc[df.i9.between('4262','42653'),'ARYTHM'] =1
    df.loc[df.i9.between('4266','42689'),'ARYTHM'] =1
    df.loc[df.i9.between('V450','V4509'),'ARYTHM'] =1
    df.loc[df.i9.between('V533','V5339'),'ARYTHM'] =1
    ##VALVE
    df.loc[df.i9=='3979','VALVE'] =1
    df.loc[df.i9=='V422','VALVE'] =1
    df.loc[df.i9=='V423','VALVE'] =1
    df.loc[df.i9.between('09320','09324'),'VALVE'] =1
    df.loc[df.i9.between('3940','3971'),'VALVE'] =1
    df.loc[df.i9.between('4240','42499'),'VALVE'] =1
    df.loc[df.i9.between('7463','7466'),'VALVE'] =1
    ##PULMCIRC
    df.loc[df.i9=='4179','PULMCIRC'] =1
    df.loc[df.i9.between('41511','41519'),'PULMCIRC'] =1
    df.loc[df.i9.between('4160','4169'),'PULMCIRC'] =1
    ##PERIVASC
    df.loc[df.i9.between('4400','4409'),'PERIVASC'] =1
    df.loc[df.i9.between('44100','4419'),'PERIVASC'] =1
    df.loc[df.i9.between('4420','4429'),'PERIVASC'] =1
    df.loc[df.i9.between('4431','4439'),'PERIVASC'] =1
    df.loc[df.i9.between('44421','44422'),'PERIVASC'] =1
    df.loc[df.i9=='4471','PERIVASC'] =1
    df.loc[df.i9=='449','PERIVASC'] =1
    df.loc[df.i9=='5571','PERIVASC'] =1
    df.loc[df.i9=='5579','PERIVASC'] =1
    df.loc[df.i9=='V434','PERIVASC'] =1
    ##HTN
    df.loc[df.i9=='4011','HTN'] =1
    df.loc[df.i9=='4019','HTN'] =1
    df.loc[df.i9.between('64200','64204'),'HTN'] =1
    #HTNCX
    df.loc[df.i9=='4010','HTNCX'] =1
    df.loc[df.i9=='4372','HTNCX'] =1
    #HTNPREG
    df.loc[df.i9.between('64220','64224'),'HTNPREG'] =1
    #HTNWOCHF
    df.loc[df.i9=='40200','HTNWOCHF'] =1
    df.loc[df.i9=='40210','HTNWOCHF'] =1
    df.loc[df.i9=='40290','HTNWOCHF'] =1
    df.loc[df.i9=='40509','HTNWOCHF'] =1
    df.loc[df.i9=='40519','HTNWOCHF'] =1
    df.loc[df.i9=='40599','HTNWOCHF'] =1
    ##HTNWCHF
    df.loc[df.i9=='40201','HTNWCHF'] =1
    df.loc[df.i9=='40211','HTNWCHF'] =1
    df.loc[df.i9=='40291','HTNWCHF'] =1
    #HRENWORF
    df.loc[df.i9=='40300','HRENWORF'] =1
    df.loc[df.i9=='40310','HRENWORF'] =1
    df.loc[df.i9=='40390','HRENWORF'] =1
    df.loc[df.i9=='40501','HRENWORF'] =1
    df.loc[df.i9=='40511','HRENWORF'] =1
    df.loc[df.i9=='40591','HRENWORF'] =1
    df.loc[df.i9.between('64210','64214'),'HRENWORF'] =1
    #HRENWRF
    df.loc[df.i9=='40301','HRENWRF'] =1
    df.loc[df.i9=='40311','HRENWRF'] =1
    df.loc[df.i9=='40391','HRENWRF'] =1
    #HHRWOHRF
    df.loc[df.i9=='40400','HHRWOHRF'] =1
    df.loc[df.i9=='40410','HHRWOHRF'] =1
    df.loc[df.i9=='40490','HHRWOHRF'] =1
    #HHRWCHF
    df.loc[df.i9=='40401','HHRWCHF'] =1
    df.loc[df.i9=='40411','HHRWCHF'] =1
    df.loc[df.i9=='40491','HHRWCHF'] =1
    #HHRWRF
    df.loc[df.i9=='40402','HHRWRF'] =1
    df.loc[df.i9=='40412','HHRWRF'] =1
    df.loc[df.i9=='40492','HHRWRF'] =1
    #HHRWHRF
    df.loc[df.i9=='40403','HHRWHRF'] =1
    df.loc[df.i9=='40413','HHRWHRF'] =1
    df.loc[df.i9=='40493','HHRWHRF'] =1
    #OHTNPREG
    df.loc[df.i9.between('64270','64274'),'OHTNPREG'] =1
    df.loc[df.i9.between('64290','64294'),'OHTNPREG'] =1
    #PARA
    df.loc[df.i9.between('3420','3449'),'PARA'] =1
    df.loc[df.i9.between('43820','43853'),'PARA'] =1
    df.loc[df.i9=='78072','PARA'] =1
    #NEURO
    df.loc[df.i9.between('3300','3319'),'NEURO'] =1
    df.loc[df.i9=='3320','NEURO'] =1
    df.loc[df.i9=='3334','NEURO'] =1
    df.loc[df.i9=='3335','NEURO'] =1
    df.loc[df.i9=='3337','NEURO'] =1
    df.loc[df.i9=='33371','NEURO'] =1
    df.loc[df.i9=='33372','NEURO'] =1
    df.loc[df.i9=='33379','NEURO'] =1
    df.loc[df.i9=='33385','NEURO'] =1
    df.loc[df.i9=='33394','NEURO'] =1
    df.loc[df.i9.between('3340','3359'),'NEURO'] =1
    df.loc[df.i9=='3380','NEURO'] =1
    df.loc[df.i9=='340','NEURO'] =1
    df.loc[df.i9.between('3411','3419'),'NEURO'] =1
    df.loc[df.i9.between('34500','34511'),'NEURO'] =1
    df.loc[df.i9.between('3452','3453'),'NEURO'] =1
    df.loc[df.i9.between('34540','34591'),'NEURO'] =1
    df.loc[df.i9.between('34700','34701'),'NEURO'] =1
    df.loc[df.i9.between('34710','34711'),'NEURO'] =1
    df.loc[df.i9=='3483','NEURO'] =1
    df.loc[df.i9.between('64940','64944'),'NEURO'] =1
    df.loc[df.i9=='7687','NEURO'] =1
    df.loc[df.i9.between('76870','76873'),'NEURO'] =1
    df.loc[df.i9=='7803','NEURO'] =1
    df.loc[df.i9=='78031','NEURO'] =1
    df.loc[df.i9=='78032','NEURO'] =1
    df.loc[df.i9=='78033','NEURO'] =1
    df.loc[df.i9=='78039','NEURO'] =1
    df.loc[df.i9=='78097','NEURO'] =1
    df.loc[df.i9=='7843','NEURO'] =1
    #CHRNLUNG
    df.loc[df.i9.between('490','4928'),'CHRNLUNG'] =1
    df.loc[df.i9.between('49300','49392'),'CHRNLUNG'] =1
    df.loc[df.i9.between('494','4941'),'CHRNLUNG'] =1
    df.loc[df.i9.between('4950','505'),'CHRNLUNG'] =1
    df.loc[df.i9=='5064','CHRNLUNG'] =1
    #DM
    df.loc[df.i9.between('25000','25033'),'DM'] =1
    df.loc[df.i9.between('64800','64804'),'DM'] =1
    df.loc[df.i9.between('24900','24931'),'DM'] =1
    #DMCX
    df.loc[df.i9.between('25040','25093'),'DMCX'] =1
    df.loc[df.i9=='7751','DMCX'] =1
    df.loc[df.i9.between('24940','24991'),'DMCX'] =1
    #HYPOTHY
    df.loc[df.i9.between('243','2442'),'HYPOTHY'] =1
    df.loc[df.i9=='2448','HYPOTHY'] =1
    df.loc[df.i9=='2449','HYPOTHY'] =1
    #RENLFAIL
    df.loc[df.i9=='585','RENLFAIL'] =1
    df.loc[df.i9=='5853','RENLFAIL'] =1
    df.loc[df.i9=='5854','RENLFAIL'] =1
    df.loc[df.i9=='5855','RENLFAIL'] =1
    df.loc[df.i9=='5856','RENLFAIL'] =1
    df.loc[df.i9=='5859','RENLFAIL'] =1
    df.loc[df.i9=='586','RENLFAIL'] =1
    df.loc[df.i9=='V420','RENLFAIL'] =1
    df.loc[df.i9=='V451','RENLFAIL'] =1
    df.loc[df.i9=='V568','RENLFAIL'] =1
    df.loc[df.i9.between('V560','V5632'),'RENLFAIL'] =1
    df.loc[df.i9.between('V4511','V4512'),'RENLFAIL'] =1
    #LIVER
    df.loc[df.i9=='4560','LIVER'] =1
    df.loc[df.i9=='4561','LIVER'] =1
    df.loc[df.i9=='45620','LIVER'] =1
    df.loc[df.i9=='45621','LIVER'] =1
    df.loc[df.i9=='5710','LIVER'] =1
    df.loc[df.i9=='5713','LIVER'] =1
    df.loc[df.i9=='5718','LIVER'] =1
    df.loc[df.i9=='5719','LIVER'] =1
    df.loc[df.i9=='5723','LIVER'] =1
    df.loc[df.i9=='5728','LIVER'] =1
    df.loc[df.i9=='5735','LIVER'] =1
    df.loc[df.i9=='V427','LIVER'] =1
    df.loc[df.i9=='07022','LIVER'] =1
    df.loc[df.i9=='07023','LIVER'] =1
    df.loc[df.i9=='07032','LIVER'] =1
    df.loc[df.i9=='07033','LIVER'] =1
    df.loc[df.i9=='07044','LIVER'] =1
    df.loc[df.i9=='07054','LIVER'] =1
    df.loc[df.i9=='5712','LIVER'] =1
    df.loc[df.i9=='5715','LIVER'] =1
    df.loc[df.i9=='576','LIVER'] =1
    df.loc[df.i9.between('57140','57149'),'RENLFAIL'] =1
    #LYMPH
    df.loc[df.i9.between('20000','20238'),'LYMPH'] =1
    df.loc[df.i9.between('20250','20301'),'LYMPH'] =1
    df.loc[df.i9=='2386','LYMPH'] =1
    df.loc[df.i9=='2733','LYMPH'] =1
    df.loc[df.i9.between('20302','20382'),'LYMPH'] =1
    df.loc[df.i9.between('20240','20249'),'LYMPH'] =1
    df.loc[df.i9.between('204','20492'),'LYMPH'] =1
    df.loc[df.i9.between('205','20592'),'LYMPH'] =1
    df.loc[df.i9.between('206','20692'),'LYMPH'] =1
    df.loc[df.i9.between('207','20782'),'LYMPH'] =1
    df.loc[df.i9.between('208','20892'),'LYMPH'] =1
    #METS
    df.loc[df.i9.between('1960','1991'),'METS'] =1
    df.loc[df.i9.between('20970','20975'),'METS'] =1
    df.loc[df.i9=='20979','METS'] =1
    df.loc[df.i9=='78951','METS'] =1
    #TUMOR
    df.loc[df.i9.between('1400','1729'),'TUMOR'] =1
    df.loc[df.i9.between('1740','1759'),'TUMOR'] =1
    df.loc[df.i9.between('179','1958'),'TUMOR'] =1
    df.loc[df.i9.between('20900','20924'),'TUMOR'] =1
    df.loc[df.i9.between('20925','2093'),'TUMOR'] =1
    df.loc[df.i9.between('20930','20936'),'TUMOR'] =1
    df.loc[df.i9.between('25801','25803'),'TUMOR'] =1
    #ARTH
    df.loc[df.i9=='7010','ARTH'] =1
    df.loc[df.i9=='725','ARTH'] =1
    df.loc[df.i9.between('7100','7109'),'ARTH'] =1
    df.loc[df.i9.between('7140','7149'),'ARTH'] =1
    df.loc[df.i9.between('7200','7209'),'ARTH'] =1
    #COAG
    df.loc[df.i9.between('2860','2869'),'COAG'] =1
    df.loc[df.i9=='2871','COAG'] =1
    df.loc[df.i9.between('2873','2875'),'COAG'] =1
    df.loc[df.i9.between('64930','64934'),'COAG'] =1
    df.loc[df.i9=='28984','COAG'] =1
    #OBESE
    df.loc[df.i9=='2780','OBESE'] =1
    df.loc[df.i9=='27800','OBESE'] =1
    df.loc[df.i9=='27801','OBESE'] =1
    df.loc[df.i9=='27803','OBESE'] =1
    df.loc[df.i9.between('64910','64914'),'OBESE'] =1
    df.loc[df.i9.between('V8530','V8539'),'OBESE'] =1
    df.loc[df.i9=='V854','OBESE'] =1
    df.loc[df.i9.between('V8541','V8545'),'OBESE'] =1
    df.loc[df.i9=='V8554','OBESE'] =1
    df.loc[df.i9=='79391','OBESE'] =1
    #LYTES
    df.loc[df.i9.between('2760','2769'),'LYTES'] =1
    #ANEMDEF
    df.loc[df.i9=='2859','ANEMDEF'] =1
    df.loc[df.i9.between('2801','2819'),'ANEMDEF'] =1
    df.loc[df.i9.between('28521','28529'),'ANEMDEF'] =1
    #ALCOHOL
    df.loc[df.i9.between('2910','2913'),'ALCOHOL'] =1
    df.loc[df.i9=='2915','ALCOHOL'] =1
    df.loc[df.i9=='2918','ALCOHOL'] =1
    df.loc[df.i9=='29181','ALCOHOL'] =1
    df.loc[df.i9=='29182','ALCOHOL'] =1
    df.loc[df.i9=='29189','ALCOHOL'] =1
    df.loc[df.i9=='2919','ALCOHOL'] =1
    df.loc[df.i9.between('30300','30393'),'ALCOHOL'] =1
    df.loc[df.i9.between('30500','30503'),'ALCOHOL'] =1
    #DRUG
    df.loc[df.i9=='2920','DRUG'] =1
    df.loc[df.i9=='2929','DRUG'] =1
    df.loc[df.i9.between('29282','29289'),'DRUG'] =1
    df.loc[df.i9.between('30400','30493'),'DRUG'] =1
    df.loc[df.i9.between('30520','30593'),'DRUG'] =1
    df.loc[df.i9.between('64830','64834'),'DRUG'] =1
    #PSYCH
    df.loc[df.i9.between('29500','2989'),'PSYCH'] =1
    df.loc[df.i9=='29910','PSYCH'] =1
    df.loc[df.i9=='29911','PSYCH'] =1
    #DEPRESS
    df.loc[df.i9=='3004','DEPRESS'] =1
    df.loc[df.i9=='30112','DEPRESS'] =1
    df.loc[df.i9=='3090','DEPRESS'] =1
    df.loc[df.i9=='3091','DEPRESS'] =1
    df.loc[df.i9=='311','DEPRESS'] =1
    
    df.drop('i9', axis=1, inplace=True)
    df.fillna(0, inplace=True)
    comorb_dummy = df.groupby('PatientID', as_index=False)
    
    del df
    
    df = comorb_dummy.max()
    df.reset_index(drop='True', inplace=True)
    
    del comorb_dummy
    
    #CONGESTIVE_HEART_FAILURE
    df.loc[df.CHF==1,'congestive_heart_failure'] =1
    df.loc[df.HTNWCHF==1, 'congestive_heart_failure'] =1
    df.loc[df.HHRWCHF==1, 'congestive_heart_failure'] =1
    df.loc[df.HHRWHRF==1, 'congestive_heart_failure'] =1
    #CARDIAC_ARRHYTHMIAS
    df.loc[df.ARYTHM==1,'cardiac_arrhythmias'] =1
    #VALVULAR_DISEASE
    df.loc[df.VALVE==1,'valvular_disease'] =1
    #PULMONARY_CIRCULATION
    df.loc[df.PULMCIRC==1,'pulmonary_circulation'] =1
    #PERIPHERAL_VASCULAR
    df.loc[df.PERIVASC==1,'peripheral_vascular'] =1
    #HYPERTENSION
    df.loc[df.HTN==1,'hypertension'] =1
    df.loc[df.HTNCX==1, 'hypertension'] =1
    df.loc[df.HTNPREG==1, 'hypertension'] =1
    df.loc[df.HTNWOCHF==1, 'hypertension'] =1
    df.loc[df.HTNWCHF==1, 'hypertension'] =1
    df.loc[df.HRENWORF==1, 'hypertension'] =1
    df.loc[df.HRENWRF==1, 'hypertension'] =1
    df.loc[df.HHRWOHRF==1, 'hypertension'] =1
    df.loc[df.HHRWCHF==1, 'hypertension'] =1
    df.loc[df.HHRWRF==1, 'hypertension'] =1
    df.loc[df.HHRWHRF==1, 'hypertension'] =1
    df.loc[df.OHTNPREG==1, 'hypertension'] =1
    #PARALYSIS
    df.loc[df.PARA==1,'paralysis'] =1
    #OTHER_NEUROLOGICAL
    df.loc[df.NEURO==1,'other_neurological'] =1
    #CHRONIC_PULMONARY
    df.loc[df.CHRNLUNG==1,'chronic_pulmonary'] =1
    #HYPOTHYROIDISM
    df.loc[df.HYPOTHY==1,'hypothyroidism'] =1
    #DIABETES
    df.loc[df.DM==1,'diabetes'] =1
    df.loc[df.DMCX==1,'diabetes'] =1
    #RENAL_FAILURE
    df.loc[df.RENLFAIL==1,'renal_failure'] =1
    df.loc[df.HRENWRF==1, 'renal_failure'] =1
    df.loc[df.HHRWRF==1, 'renal_failure'] =1
    df.loc[df.HHRWHRF==1, 'renal_failure'] =1
    #LIVER_DISEASE
    df.loc[df.LIVER==1,'liver_disease'] =1
    #LYMPHOMA
    df.loc[df.LYMPH==1,'lymphoma'] =1
    #METASTATIC_CANCER
    df.loc[df.METS==1,'metastatic_cancer'] =1
    #SOLID_TUMOR
    df.loc[df.METS==1,'solid_tumor'] =1
    df.loc[df.TUMOR==1,'solid_tumor'] =1
    #RHEUMATOID_ARTHRITIS
    df.loc[df.ARTH==1,'rheumatoid_arthritis'] =1
    #COAGULOPATHY
    df.loc[df.COAG==1,'coagulopathy'] =1
    #OBESITY
    df.loc[df.OBESE==1,'obesity'] =1
    #FLUID_ELECTROLYTE
    df.loc[df.LYTES==1,'fluid_electrolyte'] =1
    #DEFICIENCY_ANEMIAS
    df.loc[df.ANEMDEF==1,'deficiency_anemias'] =1
    #ALCOHOL_ABUSE
    df.loc[df.ALCOHOL==1,'alcohol_abuse'] =1
    #DRUG_ABUSE
    df.loc[df.DRUG==1,'drug_abuse'] =1
    #PSYCHOSES
    df.loc[df.PSYCH==1,'psychoses'] =1
    #DEPRESSION
    df.loc[df.DEPRESS==1,'depression'] =1
    ##order_all
    df = df.drop(columns=['CHF','ARYTHM','VALVE','PULMCIRC','PERIVASC','HTN','HTNCX','HTNPREG','HTNWOCHF',
                          'HTNWCHF','HRENWORF','HRENWRF','HHRWOHRF','HHRWCHF','HHRWRF','HHRWHRF','OHTNPREG',
                          'PARA','NEURO','CHRNLUNG','DM','DMCX','HYPOTHY','RENLFAIL','LIVER',
                          'LYMPH','METS','TUMOR','ARTH','COAG','OBESE','LYTES','ANEMDEF',
                          'ALCOHOL','DRUG','PSYCH','DEPRESS'])
    
    df.fillna(0, inplace=True)
    df = df.astype(int)
    
    return df

In [17]:
def comorbidities_grouping(df):
    
    df['chronic_pulmonary_disease'] = np.where((df['chronic_pulmonary']==1) | (df['pulmonary_circulation']==1),1,0)
    df['psychiatric'] = np.where((df['depression']==1) | (df['psychoses']==1),1,0)
    df['chronic_neurologic_diseases'] = np.where((df['paralysis']==1) | (df['other_neurological']==1),1,0)
    df['substance_abuse'] = np.where((df['drug_abuse']==1) | (df['alcohol_abuse']==1),1,0)
    df['heart_diseases'] = np.where((df['congestive_heart_failure']==1) | (df['cardiac_arrhythmias']==1) | (
                                     df['valvular_disease']),1,0)
    
    df.rename({'solid_tumor':'solid_cancer', 'hematologic_malignancy':'lymphoma'}, axis=1, inplace=True)
    
    df.drop(['chronic_pulmonary','pulmonary_circulation','depression','psychoses','paralysis',
             'other_neurological','drug_abuse','alcohol_abuse','lymphoma','cardiac_arrhythmias',
             'congestive_heart_failure','valvular_disease'], axis=1, inplace=True)
    
    return df

In [18]:
def comorbidities(df):
    
    db = df['db'][0]
    
    if db == 'eicu':

        comorbidities1 = mimicsql_comorbidities_template4eicu(df)
        comorbidities2 = pasthistory_comorbidities4eicu(df)
        
        comorbidities_dummy = pd.concat([comorbidities1, comorbidities2])
        
        del comorbidities1, comorbidities2
        
        comorbidities_dummy.fillna(0, inplace=True)
        comorbidities = comorbidities_dummy.groupby('patientunitstayid', as_index=False)
        comorbidities = comorbidities.max()
        comorbidities.reset_index(drop='True', inplace=True)
        
        comorbidities_clean = comorbidities_grouping(comorbidities)
        
        del comorbidities
        
        df = pd.merge(df, comorbidities_clean, on='patientunitstayid', how='left')
        
    elif db == 'mimic':
        
        patlist = tuple(set(df['hadm_id']))
        
        conn = connection_lcp(db)
        
        comorbidities = pd.read_sql(f"""
                                SELECT
                                *
                                FROM mimiciii.elixhauser_ahrq_no_drg
                                WHERE hadm_id IN {patlist}
                                """, conn)
        
        conn.close()
        
        comorbidities['diabetes'] = np.where((comorbidities['diabetes_complicated']==1) | (
                                              comorbidities['diabetes_uncomplicated']==1),1,0)
        
        comorbidities.drop(['diabetes_complicated','diabetes_uncomplicated','aids','blood_loss_anemia',
                            'peptic_ulcer','weight_loss'], axis=1, inplace=True)
        
        comorbidities = comorbidities.drop('subject_id', axis=1)
        
        comorbidities_clean = comorbidities_grouping(comorbidities)
        
        del comorbidities
        
        df = pd.merge(df, comorbidities_clean, on='hadm_id', how='left')
        
    else:
        
        diagsparsed = diag_parser(df)
        comorbidities = mimicsql_comorbidities_template4hj23(diagsparsed)
        
        comorbidities_clean = comorbidities_grouping(comorbidities)
        
        del comorbidities
        
        df = pd.merge(df, comorbidities_clean, on='PatientID', how='left')

    return df, comorbidities_clean

In [19]:
def parse_and_concat_dbs(eicu, mimic, hj23):
    
    eicu['icu_los'] = round(eicu['unitdischargeoffset']/1440,1)
    eicu['hosp_los'] = round(eicu['hospitaldischargeoffset']/1440,1)
    eicu['hospdis_status'] = np.where(eicu['hospitaldischargestatus']=='Alive',0,1)
    eicu['gender'] = np.where(eicu['gender']=='Male','M','F')
    eicu.rename({'patientunitstayid':'patient_id'}, axis=1, inplace=True)
    eicu.drop(['unitdischargeoffset','hospitaldischargeoffset','hospitaldischargestatus'], axis=1, inplace=True)
    
    mimic.rename({'icustay_id':'patient_id', 'los':'icu_los',
                  'hospital_expire_flag':'hospdis_status'}, axis=1, inplace=True)
    mimic.drop(['intime','hadm_id','subject_id'], axis=1, inplace=True)
    
    hj23['hospdis_status'] = np.where(hj23['hosp_dis']=='ALIVE',0,1)
    hj23.rename({'PatientID':'patient_id', 'ICU_days':'icu_los'}, axis=1, inplace=True)
    hj23.drop(['PatientCode','AdmTime','hosp_dis'], axis=1, inplace=True)
    
    alldbs = pd.concat([eicu, mimic, hj23])
    
    del eicu, mimic, hj23
    
    alldbs = alldbs[['db','patient_id','gender','age','icu_los','hosp_los','hospdis_status','sis','sis_q',
                     'lactate','lactate_label','lactate_offset','alt','ast','bun','wbc','bicarbonate','calcium',
                     'chloride','creatinine','glucose','platelets','potassium','sodium','bilirubin',
                     'hr','bp','temperature','peripheral_vascular','hypertension','hypothyroidism',
                     'diabetes','renal_failure','liver_disease','metastatic_cancer','solid_cancer',
                     'rheumatoid_arthritis','coagulopathy','obesity','fluid_electrolyte','deficiency_anemias',
                     'chronic_pulmonary_disease', 'psychiatric','chronic_neurologic_diseases','substance_abuse',
                     'heart_diseases']]
    
    return alldbs

## ETL

In [28]:
%%time
""" eICU ETL """
eicu_f1 = age_los_filter('eicu')
eicu_f2 = sis(eicu_f1)
eicu_f3, eicu_lactates = lactate(eicu_f2) 
eicu_septic_allq_il, eicu_f4 = septic(eicu_f3)
eicu_f5, eicu_labs = labs(eicu_f4)
eicu_f6, eicu_feats = feats(eicu_f5)
eicu_complete, eicu_comorbidities = comorbidities(eicu_f6)
""" MIMIC ETL """
mimic_f1 = age_los_filter('mimic')
mimic_f2 = sis(mimic_f1)
mimic_f3, mimic_lactates = lactate(mimic_f2) 
mimic_septic_allq_il, mimic_f4 = septic(mimic_f3)
mimic_f5, mimic_labs = labs(mimic_f4)
mimic_f6, mimic_feats = feats(mimic_f5)
mimic_complete, mimic_comorbidities = comorbidities(mimic_f6)
""" HJ23 ETL """
hj23_f1 = age_los_filter('hj23')
hj23_f2 = sis(hj23_f1)
hj23_f3, hj23_lactates = lactate(hj23_f2) 
hj23_septic_allq_il, hj23_f4 = septic(hj23_f3)
hj23_f5, hj23_labs = labs(hj23_f4)
hj23_f6, hj23_feats = feats(hj23_f5)
hj23_complete, hj23_comorbidities = comorbidities(hj23_f6)
"""Join All DBs"""
alldbs = parse_and_concat_dbs(eicu_complete, mimic_complete, hj23_complete)

eicu patients >= 16 and ICU LOS > 1 day:  132666
eicu patients >= 16 and ICU LOS > 1 day with SIS:  109099
eicu patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission:  40261
eicu patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic:  11678
eicu patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic and
                  at 4th SIS quartile:  5107
eicu patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic and
                  at 4th SIS quartile and with normal (<2) or high (>=4) lactate:  3394
mimic patients >= 16 and ICU LOS > 1 day:  45318
mimic patients >= 16 and ICU LOS > 1 day with SIS:  45317
mimic patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission:  27638
mimic patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic:  3853
mimic patients >= 16 and ICU LOS > 1 day with SIS and lactate at ICU admission and Septic and
      

## Additions

### Hosp Mortality at 28d

In [29]:
alldbs['hospdis_status_28d'] = np.where(
    (alldbs['hosp_los']>28) & (alldbs['hospdis_status']==1), 0, alldbs['hospdis_status'])

In [45]:
#alldbs.to_csv('alldbs.csv', index=False)

In [46]:
#alldbs = pd.read_csv('alldbs.csv')

## Figures

### Lactate vs mortality

In [31]:
eicu = eicu_septic_allq_il[['patientunitstayid','hospitaldischargestatus','sis_q','lactate']]
mimic = mimic_septic_allq_il[['icustay_id','hospital_expire_flag','sis_q','lactate']]
hj23 = hj23_septic_allq_il[['PatientID','hosp_dis','sis_q','lactate']]

In [32]:
eicu.columns = ['patient','dead','sis_q','lactate']
mimic.columns = ['patient','dead','sis_q','lactate']
hj23.columns = ['patient','dead','sis_q','lactate']

In [33]:
eicu['dead'] = np.where(eicu['dead']=='Alive',0,1)

In [34]:
hj23['dead'] = np.where(hj23['dead']=='ALIVE',0,1)

In [35]:
df4figure = pd.concat([eicu,mimic,hj23])

In [36]:
df4figure['lactate_group'] = np.where(
    df4figure['lactate']<2,'normal',np.where(df4figure['lactate']<4,'intermediate','high'))

In [37]:
df4figure_groupped = df4figure.groupby('lactate_group', as_index=False).agg({'patient':'count',
                                                                             'dead':'mean'})

In [38]:
df4figure_groupped_q4 = df4figure[df4figure['sis_q']==4].groupby('lactate_group', as_index=False).agg(
                                                                                        {'patient':'count',
                                                                                         'dead':'mean'})

In [39]:
df4figure_groupped['dead'] = round(df4figure_groupped['dead']*100,2)

In [40]:
df4figure_groupped_q4['dead'] = round(df4figure_groupped_q4['dead']*100,2)

In [42]:
df4figure_groupped_q4

,lactate_group,patient,dead
0,high,2581,43.78
1,intermediate,2582,29.67
2,normal,2280,23.55


In [44]:
fig = go.Figure(data=[go.Scatter(
    x=['Normal','Intermediate','High'],
    y=[23.55, 29.67, 43.78],
    mode='markers',
    marker=dict(
        color=['#B3B3B3', '#FAA082', '#E5546C'],
        size=df4figure_groupped_q4['patient']/100*5,
    )
)])

fig.update_layout({'paper_bgcolor':'rgba(0,0,0,0)', 'plot_bgcolor':'rgba(0,0,0,0)'},
                  xaxis=dict(title='Lactate Group'),
                  yaxis=dict(title='Mortality (%)'))

fig.show()

### Lactate vs SoIS

In [57]:
eicu = alldbs[alldbs['db']=='eicu']
mimic = alldbs[alldbs['db']=='mimic']
hj23 = alldbs[alldbs['db']=='hj23']

#### Normalize SoIS

In [58]:
eicu['SoIS (normalized)']=(eicu['sis']-eicu['sis'].min())/(eicu['sis'].max()-eicu['sis'].min())
mimic['SoIS (normalized)']=(mimic['sis']-mimic['sis'].min())/(mimic['sis'].max()-mimic['sis'].min())
hj23['SoIS (normalized)']=(hj23['sis']-hj23['sis'].min())/(hj23['sis'].max()-hj23['sis'].min())

In [59]:
alldbs_normalizedsis = pd.concat([eicu,mimic,hj23])

In [60]:
#alldbs_normalizedsis.to_csv('alldbs_normalizedsis.csv', index=False)

In [61]:
df = alldbs_normalizedsis
df.rename({'lactate':'Lactate (mmol/L)', 'db':'Database'}, axis=1, inplace=True)
df.rename({'lactate_label':'Lactate Group'}, axis=1, inplace=True)
df['Lactate Group'] = np.where(df['Lactate Group']=='high','High','Normal')
fig = px.histogram(df, x="SoIS (normalized)", color="Lactate Group", 
                   color_discrete_map = {'High':'#E5546C', 'Normal':'#B3B3B3'})
fig.update_layout({'paper_bgcolor':'rgba(0,0,0,0)', 'plot_bgcolor':'rgba(0,0,0,0)'},
                  yaxis=dict(title='Nº of patients'))
#fig.write_image("images/fig1.png")
fig.show()

The analysis will continue in LactateDiscordanceAnalysis